# ECTI-P vs ΛCDM  
## Pantheon+SH0ES · BAO · RSD · KiDS · Planck 2018 (distance priors)  
### Full MCMC comparison with identical datasets and priors

This notebook presents a controlled, like-for-like comparison between the
standard flat ΛCDM cosmological model and the ECTI-P model.

Both models are tested using:
- identical datasets,
- identical likelihood definitions,
- identical priors where applicable,
- identical MCMC configurations.

The goal is not model tuning, but a fair statistical comparison based on
χ², AIC, and BIC criteria.

All chains are automatically saved to disk to prevent data loss in case
of disconnection.

# 1) Scientific context and scope

This notebook presents a **fair, parameter-matched comparison** between:
- the standard cosmological model **ΛCDM**
- the phenomenological late-time extension **ECTI-P**

## Scope (referee-proof)

- **ECTI-P is a late-time, background-only modification**: it modifies **only the background expansion** \(H(z)\) at low redshift.
- **Early-time physics is not modified**: no recombination changes, no Boltzmann solver (CLASS/CAMB), no TT/TE/EE likelihood, no CLik.
  Standard radiation content (photons + massless neutrinos) is included where required to compute early-time quantities entering \((R,\ \ell_A,\ \omega_b)\).
- **Growth is not modified by ECTI**: the RSD prediction \(f\sigma_8\) is computed using the **standard linear growth equation in GR**, driven by the model background \(H(z)\), with **no additional growth parameters**.

## Parameterization (FAIR k = 5)

Both models are sampled with the same parameter vector:
\[
\theta_5 = (H_0,\ \Omega_{m0},\ \sigma_8,\ M,\ \omega_b).
\]

ECTI-P has two internal parameters \((\beta,\ z_t)\) that are **fixed constants** (defined in Section 4).  
Therefore the comparison is strictly **same-k**: \(k=5\) vs \(k=5\), and \(\Delta \mathrm{AIC} = \Delta \mathrm{BIC} = \Delta \chi^2\).

## Datasets used

- Pantheon+SH0ES SNe Ia (with full covariance)
- DESI 2024 BAO likelihood (mean + covariance)
- RSD \(f\sigma_8\) compilation
- **KiDS compressed constraint** (Gaussian \(S_8\)-like information; **not** the full 2pt weak-lensing likelihood)
- Planck 2018 compressed distance priors \((R,\ \ell_A,\ \omega_b)\) with 3×3 covariance
- Planck 2018 lensing-only **derived 1D constraint** (optional toggle; counted as 1 datum when enabled)


## 1.1) Environment & execution policy (referee-proof)

This section defines:
- environment detection and versions
- global execution flags
- path conventions (server vs Colab)
- output folders for chains/figures/tables

Policy:
- no implicit installs unless explicitly enabled
- notebook intended to run top-to-bottom without manual intervention


In [ ]:
# ============================================================
# 1.1.1) Environment setup (JupyterLab / Hetzner / bare metal)
# ============================================================

import sys
import importlib

def require(pkg, pip_name=None):
    try:
        importlib.import_module(pkg)
        print(f"✔ {pkg} available")
    except ImportError as e:
        raise RuntimeError(
            f"Missing dependency '{pkg}'.\n"
            f"Install it ONCE on the server with:\n"
            f"  pip install {pip_name or pkg}\n"
            f"Then restart the Jupyter kernel."
        ) from e

# --- Scientific stack ---
require("numpy")
require("scipy")
require("pandas")
require("matplotlib")
require("emcee")
require("corner")

print("✔ Environment OK (JupyterLab / server mode)")


In [ ]:
# ============================================================
# 1.1.2) Global configuration + PATHS (AUTO: Colab vs JupyterLab)
#    + DESI BAO repo clone (optional if already present)
#    REFEREE-PROOF:
#      - Prefer repo-relative paths (data/rsd/rsd.csv) for portability
#      - Keep safe fallbacks for legacy layouts
# ============================================================

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emcee

np.random.seed(42)

# -----------------------------
# Detect environment
# -----------------------------
IS_COLAB = ("COLAB_GPU" in os.environ) or ("/content" in os.getcwd()) or ("google.colab" in sys.modules)
print(f"✔ Environment detected: {'COLAB' if IS_COLAB else 'JUPYTER / SERVER'}")
print(f"cwd = {os.getcwd()}")

# -----------------------------
# Constants
# -----------------------------
c_light = 299792.458  # km/s

# -----------------------------
# KiDS toggle + constants (so Cell #2 never crashes)
# -----------------------------
USE_KIDS   = True
KIDS_MEAN  = 0.766
KIDS_SIGMA = 0.020

# -----------------------------
# MCMC defaults (short-run safe)
# -----------------------------
N_WALKERS = 16
N_BURN    = 2000
N_STEPS   = 10000
RUN_MCMC  = False

# -----------------------------
# Helper: pick first existing path
# -----------------------------
def _first_existing(paths):
    for p in paths:
        try:
            if p and os.path.exists(p):
                return p
        except Exception:
            pass
    return None

CWD = os.getcwd()

# -----------------------------
# Paths: prefer repo-relative layout
#   - Canonical repo layout:
#       data/rsd/rsd.csv
#       (Pantheon files may be auto-downloaded to CWD or /content)
# -----------------------------
if IS_COLAB:
    # Pantheon goes to /content by default in Colab
    PATH_SN_DAT = "/content/Pantheon+SH0ES.dat"
    PATH_SN_COV = "/content/Pantheon+SH0ES_STAT+SYS.cov"
else:
    # Jupyter/server: default to current project directory
    PATH_SN_DAT = os.path.join(CWD, "Pantheon+SH0ES.dat")
    PATH_SN_COV = os.path.join(CWD, "Pantheon+SH0ES_STAT+SYS.cov")

# RSD: canonical path in repo + safe fallbacks
RSD_CANDIDATES = [
    os.path.join(CWD, "data", "rsd", "rsd.csv"),  # canonical GitHub layout
    os.path.join(CWD, "rsd.csv"),                 # legacy fallback (old layout)
    "/content/data/rsd/rsd.csv",                   # possible Colab clone layout
    "/content/rsd.csv",                            # legacy Colab fallback
]

# If on Colab, also try /content/<repo>/data/rsd/rsd.csv
if IS_COLAB and os.path.isdir("/content"):
    try:
        for name in os.listdir("/content"):
            RSD_CANDIDATES.append(os.path.join("/content", name, "data", "rsd", "rsd.csv"))
    except Exception:
        pass

PATH_RSD = _first_existing(RSD_CANDIDATES)

# -----------------------------
# Pantheon+SH0ES (official DataRelease) — auto-download if missing
# SAFE: triggers only if the local files are absent
# -----------------------------
PANTHEON_DAT_URL = "https://raw.githubusercontent.com/PantheonPlusSH0ES/DataRelease/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES.dat"
PANTHEON_COV_URL = "https://raw.githubusercontent.com/PantheonPlusSH0ES/DataRelease/main/Pantheon%2B_Data/4_DISTANCES_AND_COVAR/Pantheon%2BSH0ES_STAT%2BSYS.cov"

def _download_if_missing(url, dest_path, label):
    if os.path.isfile(dest_path) and (os.path.getsize(dest_path) > 0):
        return
    os.makedirs(os.path.dirname(dest_path) or ".", exist_ok=True)
    print(f"▶ Downloading {label} → {dest_path}")
    try:
        import urllib.request
        urllib.request.urlretrieve(url, dest_path)
    except Exception as e:
        raise RuntimeError(
            f"Failed to download {label}.\n"
            f"URL : {url}\n"
            f"DEST: {dest_path}\n"
            f"Error: {e}"
        )

_download_if_missing(PANTHEON_DAT_URL, PATH_SN_DAT, "Pantheon+SH0ES.dat")
_download_if_missing(PANTHEON_COV_URL, PATH_SN_COV, "Pantheon+SH0ES_STAT+SYS.cov")

# -----------------------------
# DESI BAO data repo (CobayaSampler/bao_data)
# On Colab we can clone into /content; on server into ./bao_data
# -----------------------------
BAO_REPO_DIR = "/content/bao_data" if IS_COLAB else os.path.join(CWD, "bao_data")

if not os.path.isdir(BAO_REPO_DIR):
    print(f"▶ Cloning https://github.com/CobayaSampler/bao_data.git into {BAO_REPO_DIR} ...")
    rc = os.system(f"git clone https://github.com/CobayaSampler/bao_data.git {BAO_REPO_DIR}")
    if rc != 0:
        raise RuntimeError("git clone failed (is git installed / internet available?)")
else:
    print(f"✔ bao_data repo already present: {BAO_REPO_DIR}")

DESI_MEAN_PATH = os.path.join(BAO_REPO_DIR, "desi_2024_gaussian_bao_ALL_GCcomb_mean.txt")
DESI_COV_PATH  = os.path.join(BAO_REPO_DIR, "desi_2024_gaussian_bao_ALL_GCcomb_cov.txt")

# -----------------------------
# Hard check: required files
# -----------------------------
if PATH_RSD is None:
    raise FileNotFoundError(
        "RSD file not found. Expected one of:\n"
        "- ./data/rsd/rsd.csv (recommended, GitHub layout)\n"
        "- ./rsd.csv (legacy)\n"
        "- /content/<repo>/data/rsd/rsd.csv (Colab clone)\n"
        "- /content/rsd.csv (legacy Colab)\n"
    )

required_paths = {
    "PATH_SN_DAT": PATH_SN_DAT,
    "PATH_SN_COV": PATH_SN_COV,
    "PATH_RSD": PATH_RSD,
    "DESI_MEAN": DESI_MEAN_PATH,
    "DESI_COV": DESI_COV_PATH,
}

missing = [k for k, v in required_paths.items() if not os.path.exists(v)]

print("\n--- Required paths ---")
for k, v in required_paths.items():
    print(f"{k} -> {v} | exists={os.path.exists(v)}")

if missing:
    raise FileNotFoundError(
        "Missing required file(s):\n- " + "\n- ".join([f"{k}: {required_paths[k]}" for k in missing]) +
        "\n\nFix:\n"
        "- On COLAB: ensure the repo is cloned (so ./data/rsd/rsd.csv exists) or upload rsd.csv to /content.\n"
        "- On Jupyter/server: run from the repo root (so ./data/rsd/rsd.csv exists), or place files accordingly.\n"
    )

print("\n✔ Required files found (SN, RSD, DESI BAO mean+cov)")

In [ ]:
# ============================================================
# 1.1.3) LOCAL OUTPUT PATHS (JupyterLab) + RUN NUMBERING + RESUME
# ============================================================

import os
import re
import time
import json
import shutil
from pathlib import Path

# --- Base output folder (LOCAL, no Drive) ---
BASE_RUN_DIR = os.path.abspath("ECTI_runs")
CHAINS_DIR   = os.path.join(BASE_RUN_DIR, "chains_sigma8free")
FIG_DIR      = os.path.join(CHAINS_DIR, "figures")
TAB_DIR      = os.path.join(CHAINS_DIR, "tables")
META_DIR     = os.path.join(CHAINS_DIR, "_meta")

for d in [BASE_RUN_DIR, CHAINS_DIR, FIG_DIR, TAB_DIR, META_DIR]:
    os.makedirs(d, exist_ok=True)

# --- Behaviour flags ---
RESUME_IF_INCOMPLETE = True     # reprend le dernier run si backend.iteration < N_STEPS
WRITE_LATEST_COPY    = True     # copie pratique vers *_latest.h5 après chaque run

def _list_runs(model_tag: str):
    """
    Retourne la liste triée des fichiers: CHAINS_DIR/{model_tag}_runXXX.h5
    """
    pat = re.compile(rf"^{re.escape(model_tag)}_run(\d+)\.h5$")
    found = []
    for fn in os.listdir(CHAINS_DIR):
        m = pat.match(fn)
        if m:
            found.append((int(m.group(1)), os.path.join(CHAINS_DIR, fn)))
    found.sort(key=lambda x: x[0])
    return [p for _, p in found]

def latest_run_path(model_tag: str):
    runs = _list_runs(model_tag)
    return runs[-1] if runs else None

def next_run_path(model_tag: str):
    runs = _list_runs(model_tag)
    if not runs:
        return os.path.join(CHAINS_DIR, f"{model_tag}_run001.h5")
    last = runs[-1]
    m = re.search(r"run(\d+)\.h5$", last)
    last_idx = int(m.group(1)) if m else 0
    return os.path.join(CHAINS_DIR, f"{model_tag}_run{last_idx+1:03d}.h5")

def copy_as_latest(src_h5: str, model_tag: str):
    """
    Copie le fichier run vers {model_tag}_latest.h5 (pratique pour lecture rapide).
    """
    if not WRITE_LATEST_COPY:
        return None
    dst = os.path.join(CHAINS_DIR, f"{model_tag}_latest.h5")
    shutil.copy2(src_h5, dst)
    return dst

print("✔ Jupyter local outputs ready")
print("  BASE_RUN_DIR:", BASE_RUN_DIR)
print("  CHAINS_DIR  :", CHAINS_DIR)
print("  FIG_DIR     :", FIG_DIR)
print("  TAB_DIR     :", TAB_DIR)

# 2) Observational datasets and validation
This section loads all observational datasets used in the analysis
(Pantheon+ SN Ia, BAO, RSD, KiDS, Planck compressed priors, and Planck lensing),
and performs strict shape and consistency checks before any likelihood evaluation.


In [ ]:
# ============================================================
# 2.1) Data loading and validation (SN, DESI BAO, RSD, KiDS, CMB)
#   - DESI cov loaded with np.loadtxt (NO regex) to avoid misparsing
#   - Handles shapes: (12,12), (24,12), (12,24)
# ============================================================

import pandas as pd
import numpy as np
import re
import os

# ------------------------------------------------------------
# Safety: ensure DESI_MEAN_PATH / DESI_COV_PATH are defined
# (prevents NameError if Cell #1.1.2 was modified or not executed)
# ------------------------------------------------------------
if ("DESI_MEAN_PATH" not in globals()) or ("DESI_COV_PATH" not in globals()):
    BAO_REPO_DIR = globals().get("BAO_REPO_DIR", os.path.join(os.getcwd(), "bao_data"))
    DESI_MEAN_PATH = os.path.join(BAO_REPO_DIR, "desi_2024_gaussian_bao_ALL_GCcomb_mean.txt")
    DESI_COV_PATH  = os.path.join(BAO_REPO_DIR, "desi_2024_gaussian_bao_ALL_GCcomb_cov.txt")

if not os.path.exists(DESI_MEAN_PATH):
    raise FileNotFoundError(f"DESI mean file not found: {DESI_MEAN_PATH}")
if not os.path.exists(DESI_COV_PATH):
    raise FileNotFoundError(f"DESI cov file not found: {DESI_COV_PATH}")

# ------------------------------------------------------------
# Supernovae: Pantheon+SH0ES
# ------------------------------------------------------------
sn = np.genfromtxt(PATH_SN_DAT, dtype=None, names=True, encoding="utf-8")

required_cols = ["zHD", "m_b_corr"]
for col in required_cols:
    if col not in sn.dtype.names:
        raise ValueError(
            f"SN file missing required column '{col}'. Available columns: {sn.dtype.names}"
        )

z_SN  = sn["zHD"].astype(float)
mu_SN = sn["m_b_corr"].astype(float)
N_SN  = len(z_SN)

print("=== Supernovae (Pantheon+SH0ES) ===")
print(f"N_SN     = {N_SN}")
print(f"z range   = [{z_SN.min():.6f}, {z_SN.max():.6f}]")
print("observable= m_b_corr (stored in mu_SN for legacy naming)")
print("")

cov_flat = np.loadtxt(PATH_SN_COV)
N_cov = int(cov_flat[0])
C_SN = cov_flat[1:].reshape(N_cov, N_cov)

if N_cov != N_SN:
    raise ValueError(f"SN covariance mismatch: N_cov={N_cov}, N_SN={N_SN}")

Cinv_SN = np.linalg.inv(C_SN)

print("=== SN covariance ===")
print(f"C_SN shape   = {C_SN.shape}")
print(f"Cinv_SN shape= {Cinv_SN.shape}")
print("")

# ------------------------------------------------------------
# DESI DR1 BAO mean + covariance (bao_data)
# ------------------------------------------------------------

def _load_mean_file_robust(path):
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            s = line.strip()
            if (not s) or s.startswith("#"):
                continue
            nums = re.findall(r"[-+]?\d*\.\d+|[-+]?\d+", s)
            if len(nums) >= 2:
                rows.append([float(nums[0]), float(nums[1])])
    if len(rows) == 0:
        raise RuntimeError(f"Mean file unreadable/empty: {path}")
    return np.array(rows, dtype=float)

mean_arr = _load_mean_file_robust(DESI_MEAN_PATH)  # (N,2): z, obs
z_BAO   = mean_arr[:, 0].astype(float)
BAO_obs = mean_arr[:, 1].astype(float)
N_BAO_VEC = BAO_obs.size

# --- Load cov with np.loadtxt (reliable)
raw_cov = np.loadtxt(DESI_COV_PATH, dtype=float)

print("=== DESI DR1 BAO (mean + cov) ===")
print(f"Vector dim = {N_BAO_VEC}")
print(f"raw_cov shape from np.loadtxt = {raw_cov.shape}")

# --- Normalize to a 12x12 covariance
if raw_cov.shape == (N_BAO_VEC, N_BAO_VEC):
    BAO_cov = raw_cov

elif raw_cov.shape == (2*N_BAO_VEC, N_BAO_VEC):
    # two blocks stacked vertically: try top or bottom
    C_top = raw_cov[:N_BAO_VEC, :]
    C_bot = raw_cov[N_BAO_VEC:, :]

    # choose the one with positive diagonal and more PSD-ish
    def _score(C):
        C = 0.5*(C + C.T)
        dmin = float(np.min(np.diag(C)))
        eigmin = float(np.min(np.linalg.eigvalsh(C)))
        return (dmin, eigmin)

    s_top = _score(C_top)
    s_bot = _score(C_bot)
    BAO_cov = C_top if (s_top[0] > s_bot[0]) else C_bot
    print(f"cov stacked (24x12): chose {'TOP' if BAO_cov is C_top else 'BOTTOM'} block | scores top={s_top}, bot={s_bot}")

elif raw_cov.shape == (N_BAO_VEC, 2*N_BAO_VEC):
    # two blocks side-by-side: try left or right
    C_left = raw_cov[:, :N_BAO_VEC]
    C_right = raw_cov[:, N_BAO_VEC:]

    def _score(C):
        C = 0.5*(C + C.T)
        dmin = float(np.min(np.diag(C)))
        eigmin = float(np.min(np.linalg.eigvalsh(C)))
        return (dmin, eigmin)

    s_left = _score(C_left)
    s_right = _score(C_right)
    BAO_cov = C_left if (s_left[0] > s_right[0]) else C_right
    print(f"cov wide (12x24): chose {'LEFT' if BAO_cov is C_left else 'RIGHT'} block | scores left={s_left}, right={s_right}")

else:
    raise RuntimeError(
        f"Unexpected DESI cov shape {raw_cov.shape}. "
        f"Expected (12,12), (24,12) or (12,24) for dim={N_BAO_VEC}."
    )

# Symmetrize + strict diagnostics
BAO_cov = 0.5 * (BAO_cov + BAO_cov.T)
diag = np.diag(BAO_cov)
eig = np.linalg.eigvalsh(BAO_cov)

print("--- DESI_BAO_cov diagnostics ---")
print(f"min diag     = {float(np.min(diag)):.6e}")
print(f"median diag  = {float(np.median(diag)):.6e}")
print(f"min eig      = {float(np.min(eig)):.6e}")
print(f"max eig      = {float(np.max(eig)):.6e}")

if float(np.min(diag)) <= 0:
    raise RuntimeError("DESI BAO covariance has non-positive diagonal after selection. STOP.")
if float(np.min(eig)) < -1e-10:
    raise RuntimeError("DESI BAO covariance is not PSD (min eigenvalue too negative). STOP.")

BAO_inv_cov = np.linalg.inv(BAO_cov)
print("✔ DESI BAO covariance selected + inverted successfully")
print("")

# Infer row-kind by repetition in z (DM then DH; singleton -> DV)
def _row_kind(z_eff):
    z = np.asarray(z_eff, dtype=float)
    n = len(z)
    kind = []
    for i in range(n):
        prev_same = (i > 0 and np.isclose(z[i], z[i-1], rtol=0, atol=1e-12))
        next_same = (i < n-1 and np.isclose(z[i], z[i+1], rtol=0, atol=1e-12))
        if next_same and not prev_same:
            kind.append("DM_over_rd")
        elif prev_same and not next_same:
            kind.append("DH_over_rd")
        else:
            kind.append("DV_over_rd")
    return kind

BAO_row_kind = _row_kind(z_BAO)

print(pd.Series(BAO_row_kind).value_counts().to_string())
print("First 12 rows (z, kind, obs):")
for i in range(min(12, N_BAO_VEC)):
    print(f"  i={i:02d}  z={z_BAO[i]:.6f}  {BAO_row_kind[i]:10s}  obs={BAO_obs[i]:.6f}")
print("✔ DESI BAO loaded + invcov ready")
print("")

# ------------------------------------------------------------
# RSD (your existing loader)
# ------------------------------------------------------------
rsd_df = pd.read_csv(PATH_RSD)

required_cols = ["z", "fs8", "err"]
for col in required_cols:
    if col not in rsd_df.columns:
        raise ValueError(
            f"RSD file missing required column '{col}'. Columns found: {list(rsd_df.columns)}"
        )

z_RSD   = rsd_df["z"].values.astype(float)
fs8_obs = rsd_df["fs8"].values.astype(float)
fs8_err = rsd_df["err"].values.astype(float)

N_RSD = len(z_RSD)

print("=== RSD data ===")
print(f"N_RSD   = {N_RSD}")
print(f"z range = [{z_RSD.min():.6f}, {z_RSD.max():.6f}]")
print("")

# ------------------------------------------------------------
# KiDS (Gaussian prior on S8)
# ------------------------------------------------------------
if USE_KIDS:
    print("=== KiDS prior ===")
    print(f"S8 = {KIDS_MEAN} ± {KIDS_SIGMA}")
    print("")

# ------------------------------------------------------------
# CMB compressed priors (already loaded elsewhere in your notebook)
# Here: only the shape/availability check to avoid silent None
# ------------------------------------------------------------
if "PLANCK_R_MEAN" in globals() and "PLANCK_R_SIGMA" in globals():
    print("=== CMB Planck compressed ===")
    print(f"R     = {PLANCK_R_MEAN} ± {PLANCK_R_SIGMA}")
    if "PLANCK_lA_MEAN" in globals() and "PLANCK_lA_SIGMA" in globals():
        print(f"lA    = {PLANCK_lA_MEAN} ± {PLANCK_lA_SIGMA}")
    print("")
else:
    print("=== CMB Planck compressed ===")
    print("⚠ PLANCK_R_MEAN / PLANCK_R_SIGMA not found in globals (check your Planck cell order).")
    print("")


“If this section runs without error, all datasets are loaded, validated, and ready for likelihood evaluation.”

# 3) ΛCDM background model (H(z), distances, SN μ(z), BAO DV/rs)

This section defines the **flat ΛCDM background cosmology** used as our baseline model.

We implement:
- the dimensionless expansion rate \(E(z)=H(z)/H_0\),
- the Hubble rate \(H(z)\),
- comoving distance \(D_C(z)\),
- luminosity distance \(D_L(z)\),
- supernova distance modulus \(\mu(z)\),
- BAO volume distance \(D_V(z)\) and the observable \(D_V(z)/r_s\).

These functions are written in a **single, consolidated block** and are used consistently
throughout the notebook for both:
- likelihood evaluation (χ²),
- plotting and diagnostics.

> Important: this block must remain unique in the notebook (no redefinitions later),
to guarantee reproducibility and prevent hidden inconsistencies.

In [ ]:
# ============================================================
# 3.1) ΛCDM background model (H(z), distances, SN μ(z), BAO DV/rs)
#    ROBUST: scalar + array + defines ALL functions used later
#    UPDATED: BAO uses computed r_s(z*) via wb (baryon density)
# ============================================================

import numpy as np
from scipy.integrate import quad

_trap = getattr(np, "trapezoid", np.trapz)
c_light = 299792.458

def _wb_default():
    # fallback referee-proof: if wb not provided in theta, we take WB_FIXED if it exists
    if "WB_FIXED" in globals():
        try:
            return float(globals()["WB_FIXED"])
        except Exception:
            pass
    # safe fallback
    return 0.02237

def _unpack_lcdm_theta(theta):
    """
    Robust unpack for LCDM vectors used across the notebook.

    Accepts common signatures:
      - (H0, Om0, M)
      - (H0, Om0, s8, M)
      - (H0, Om0, M, wb)
      - (H0, Om0, s8, M, wb)

    Returns: (H0, Om0, s8_or_nan, M, wb)
    """
    th = np.asarray(theta, dtype=float).ravel()
    if th.size == 3:
        H0, Om0, M = th
        return float(H0), float(Om0), np.nan, float(M), float(_wb_default())

    if th.size == 4:
        # Ambiguous: either (H0,Om0,s8,M) OR (H0,Om0,M,wb)
        H0, Om0, a, b = th

        # Heuristic by physical ranges:
        # M ~ [-25,-15], wb ~ [0.005,0.04], sigma8 ~ [0.3,1.3]
        looks_like_M_wb = (-25.0 < a < -15.0) and (0.005 < b < 0.04)
        looks_like_s8_M = (0.3 < a < 1.3) and (-25.0 < b < -15.0)

        if looks_like_M_wb and (not looks_like_s8_M):
            M, wb = a, b
            return float(H0), float(Om0), np.nan, float(M), float(wb)

        # default to (H0,Om0,s8,M)
        s8, M = a, b
        return float(H0), float(Om0), float(s8), float(M), float(_wb_default())

    if th.size == 5:
        H0, Om0, s8, M, wb = th
        return float(H0), float(Om0), float(s8), float(M), float(wb)

    raise ValueError(f"LCDM theta must have size 3, 4 or 5; got {th.size}")

def get_rs_star(H0, Om0, wb):
    """
    Horizon sonore approx. au découplage (Aubourg et al. 2015 style).
    wb = Omega_b * h^2.
    """
    h = float(H0) / 100.0
    om = float(Om0) * h**2
    wb = float(wb)
    return 147.06 * ((om / 0.1417)**-0.25) * ((wb / 0.02219)**-0.12)

def E_LCDM(z, H0, Om0):
    z = np.asarray(z, dtype=float)
    return np.sqrt(Om0 * (1.0 + z) ** 3 + (1.0 - Om0))

def H_LCDM(z, H0, Om0):
    return float(H0) * E_LCDM(z, H0, Om0)

def D_C_LCDM(z, H0, Om0, n=4000):
    z_arr = np.atleast_1d(np.asarray(z, dtype=float))
    DC = np.zeros_like(z_arr, dtype=float)
    for i, zz in enumerate(z_arr):
        if zz <= 0.0:
            DC[i] = 0.0
            continue
        z_grid = np.linspace(0.0, float(zz), int(n))
        Ez = E_LCDM(z_grid, H0, Om0)
        DC[i] = (c_light / float(H0)) * _trap(1.0 / Ez, x=z_grid)
    return DC[0] if np.isscalar(z) else DC

def D_M_LCDM(z, H0, Om0, n=4000):
    return D_C_LCDM(z, H0, Om0, n=n)

def D_L_LCDM(z, H0, Om0, n=4000):
    z = np.asarray(z, dtype=float)
    return (1.0 + z) * D_M_LCDM(z, H0, Om0, n=n)

def mu_LCDM(z, theta, n=4000):
    """
    SN distance modulus.
    Accepts theta sizes 3/4/5 (see _unpack_lcdm_theta).
    """
    H0, Om0, _s8, M, _wb = _unpack_lcdm_theta(theta)
    DL = D_L_LCDM(z, H0, Om0, n=n)
    return 5.0 * np.log10(DL) + 25.0 + M

def DV_LCDM(z, theta, n=4000):
    """
    BAO volume distance:
    D_V(z) = [ (c z D_M(z)^2) / H(z) ]^(1/3)
    Robust to theta sizes 3/4/5 (only uses H0,Om0).
    """
    H0, Om0, _s8, _M, _wb = _unpack_lcdm_theta(theta)
    z = np.asarray(z, dtype=float)
    DM = D_M_LCDM(z, H0, Om0, n=n)
    Hz = H_LCDM(z, H0, Om0)
    return ((c_light * z * DM**2) / Hz) ** (1.0/3.0)

def DV_over_rs_LCDM(z, theta, n=4000):
    """
    D_V(z) / r_s with r_s depending on wb (last parameter if provided).
    """
    H0, Om0, _s8, _M, wb = _unpack_lcdm_theta(theta)
    dv = DV_LCDM(z, theta, n=n)
    rs = get_rs_star(H0, Om0, wb)
    return dv / rs

print("✔ ΛCDM background ready: E,H, D_C/D_M/D_L, mu, DV, DV/rs (wb parsed safely)")

In [ ]:
# ============================================================
# 3.2) EARLY-TIME PHYSICS (referee-proof)
#   - Radiation included (photons + massless neutrinos, Neff=3.046)
#   - Sound horizons:
#       r_s(z*) at decoupling (z* fixed = 1089.92 by notebook design)
#       r_d at drag epoch via Eisenstein & Hu z_d fit + numeric integral
#   - Single source of truth for CMB/BAO coherence with wb
#
# Units:
#   H0 in km/s/Mpc, distances in Mpc, c in km/s.
# ============================================================

import numpy as np
from numpy import trapezoid as _trap

# Physical constants / standards
c_light = float(globals().get("c_light", 299792.458))  # km/s
T_CMB = 2.7255
N_EFF = 3.046

Z_STAR = float(globals().get("Z_STAR", 1089.92))  # decoupling (fixed by design)

def _omega_gamma_h2(Tcmb=T_CMB):
    # Photon density today: ωγ = Ωγ h^2
    return 2.469e-5 * (Tcmb / 2.7255)**4

def _omega_nu_h2(omega_g_h2, Neff=N_EFF):
    # Massless neutrinos: ων = ωγ * (7/8)*(4/11)^(4/3)*Neff
    return omega_g_h2 * (7.0/8.0) * (4.0/11.0)**(4.0/3.0) * Neff

def _E_early(z, H0, Om0, wb):
    """
    Early-time E(z) including radiation.
    Flat Universe: ΩΛ = 1 - Ωm - Ωr
    Inputs:
      wb = ωb = Ωb h^2
    """
    z = np.asarray(z, dtype=float)
    h = float(H0) / 100.0

    omega_g = _omega_gamma_h2()
    omega_nu = _omega_nu_h2(omega_g)

    Or0 = (omega_g + omega_nu) / (h*h)
    Om0 = float(Om0)
    Ol0 = 1.0 - Om0 - Or0

    E2 = Or0*(1.0+z)**4 + Om0*(1.0+z)**3 + Ol0
    if np.any(E2 <= 0.0):
        return np.full_like(z, np.nan, dtype=float)
    return np.sqrt(E2)

def _R_b_over_gamma(z, H0, wb):
    """
    R(z) = 3ρb/(4ργ) = (3Ωb)/(4Ωγ) * 1/(1+z)
    with Ωb = wb/h^2, Ωγ = ωγ/h^2
    """
    h = float(H0) / 100.0
    omega_g = _omega_gamma_h2()
    Ob0 = float(wb) / (h*h)
    Og0 = omega_g / (h*h)
    return (3.0*Ob0)/(4.0*Og0) * 1.0/(1.0+np.asarray(z, float))

def D_M_early(z, H0, Om0, wb, n=12000):
    """
    Comoving transverse distance using early-time E(z) (includes radiation).
    """
    z = float(z)
    if z <= 0.0:
        return 0.0
    zz = np.linspace(0.0, z, int(n), dtype=float)
    Ez = _E_early(zz, H0, Om0, wb)
    return (c_light / float(H0)) * float(_trap(1.0/Ez, x=zz))

def _rs_integral(z_upper, H0, Om0, wb, z_max=1.0e7, n=20000):
    """
    r_s(z_upper) = (c/H0) ∫_{z_upper}^{z_max} [ c_s(z)/c ] dz / E(z)
    where c_s/c = 1/sqrt(3(1+R(z)))
    """
    z_upper = float(z_upper)
    if z_upper <= 0.0:
        raise ValueError("z_upper must be > 0 for sound horizon integral.")
    z_max = float(z_max)

    # Log grid in (1+z) for stability
    u1 = np.log(1.0 + z_upper)
    u2 = np.log(1.0 + z_max)
    uu = np.linspace(u1, u2, int(n), dtype=float)
    zz = np.exp(uu) - 1.0

    Ez = _E_early(zz, H0, Om0, wb)
    Rz = _R_b_over_gamma(zz, H0, wb)
    cs_over_c = 1.0 / np.sqrt(3.0 * (1.0 + Rz))

    integrand = cs_over_c / Ez
    # dz = (1+z) du
    rs = (c_light / float(H0)) * float(_trap(integrand * (1.0 + zz), x=uu))
    return rs

def z_drag_EH(H0, Om0, wb):
    """
    Eisenstein & Hu (1998) fitting formula for drag redshift z_d.
    Uses ωm=Ωm h^2, ωb=wb.
    """
    h = float(H0)/100.0
    wm = float(Om0) * h*h
    wb = float(wb)

    b1 = 0.313 * (wm**(-0.419)) * (1.0 + 0.607*(wm**0.674))
    b2 = 0.238 * (wm**0.223)
    zd = 1291.0 * (wm**0.251) / (1.0 + 0.659*(wm**0.828)) * (1.0 + b1*(wb**b2))
    return float(zd)

def rs_star(H0, Om0, wb):
    return _rs_integral(Z_STAR, H0, Om0, wb)

def rd_drag(H0, Om0, wb):
    zd = z_drag_EH(H0, Om0, wb)
    return _rs_integral(zd, H0, Om0, wb)

print("✔ Early-time module ready: E_early(z), D_M_early, r_s(z*), r_d(z_d) with radiation + wb")

# 4) ECTI-P background model (H(z), distances, SN μ(z), BAO DV/rs)

This section defines the **ECTI-P** background expansion model used in this notebook.

ECTI-P is implemented as a **low-redshift modification** of the ΛCDM expansion rate:
\[
H^2_{\rm ECTI}(a) = H^2_{\Lambda{\rm CDM}}(a)\,\left[1 + \beta\,S(a; z_t)\right],
\]
where:
- \(\beta\) controls the amplitude of the late-time correction,
- \(S(a; z_t)\) is a smooth activation ("switch") function centered around \(z_t\),
- the model is designed to behave as ΛCDM at high redshift (early universe).

We implement:
- the switch function \(S(a)\),
- \(H(z)\) and \(E(z)=H/H_0\),
- comoving and luminosity distances \(D_C(z)\), \(D_L(z)\),
- SN distance modulus \(\mu(z)\),
- BAO volume distance \(D_V(z)\) and \(D_V(z)/r_s\).

This block is **the unique ECTI-P background definition** and must not be redefined later.

In [ ]:
# ============================================================
# 4.1) ECTI background model (FAIR k=5 vs ΛCDM, β & zt fixed)
#    θ5 = (H0, Om0, σ8, M, wb)
#    Backward-compatible unpack supports θ6/θ7 (legacy), but MCMC uses θ5.
#    r_s(z*) / early physics: SAME as ΛCDM (uses get_rs_star with wb).
# ============================================================

import numpy as np

# --- Fixed ECTI parameters (used when theta has no beta/zt) ---
BETA_FIXED = float(globals().get("BETA_FIXED", -0.10))
ZT_FIXED   = float(globals().get("ZT_FIXED",   0.10))

_trap = getattr(np, "trapezoid", np.trapz)

def _unpack_ecti_theta(theta):
    """
    Robust ECTI unpacker (referee-proof).
    Supported:
      θ5 = (H0, Om0, σ8, M, wb)                 [FAIR comparison mode]
      θ6 = (H0, Om0, σ8, beta, zt, M)           [legacy]
      θ7 = (H0, Om0, σ8, beta, zt, M, wb)       [legacy]
    Returns: (H0, Om0, σ8, beta, zt, M, wb)
    """
    th = np.asarray(theta, dtype=float).ravel()
    if th.size == 5:
        H0, Om0, s8, M, wb = th
        return float(H0), float(Om0), float(s8), BETA_FIXED, ZT_FIXED, float(M), float(wb)

    if th.size == 6:
        H0, Om0, s8, beta, zt, M = th
        wb = float(globals().get("WB_FIXED", 0.02237))
        return float(H0), float(Om0), float(s8), float(beta), float(zt), float(M), float(wb)

    if th.size == 7:
        H0, Om0, s8, beta, zt, M, wb = th
        return float(H0), float(Om0), float(s8), float(beta), float(zt), float(M), float(wb)

    raise ValueError(f"ECTI theta must have size 5/6/7; got {th.size}")

# Backward-compat alias used by some older cells
def _unpack_ecti(theta):
    return _unpack_ecti_theta(theta)

def E_ECTI(z, theta):
    """
    E(z) for ECTI-P (late-time-only modification, smooth, no extra hyper-params):
      E^2(z) = Om0(1+z)^3 + (1-Om0) * exp[ beta * exp(-z/zt) ]
    Properties:
      - As z >> zt: exp(-z/zt) -> 0 => DE term -> exp(0)=1 (ΛCDM recovered)
      - At z=0: DE term -> exp(beta)
    """
    H0, Om0, _s8, beta, zt, _M, _wb = _unpack_ecti_theta(theta)
    z = np.asarray(z, dtype=float)

    if zt <= 0.0:
        raise ValueError("ECTI requires zt > 0 (transition scale).")

    de_factor = np.exp(beta * np.exp(-z / zt))
    E2 = Om0 * (1.0 + z)**3 + (1.0 - Om0) * de_factor

    if np.any(E2 <= 0.0):
        return np.full_like(z, np.nan, dtype=float)

    return np.sqrt(E2)

def H_ECTI(z, theta):
    H0, *_ = _unpack_ecti_theta(theta)
    return H0 * E_ECTI(z, theta)

def D_C_ECTI(z, theta, n=4000):
    H0, *_ = _unpack_ecti_theta(theta)
    z_arr = np.atleast_1d(np.asarray(z, dtype=float))
    DC = np.zeros_like(z_arr, dtype=float)

    for i, zi in enumerate(z_arr):
        if zi <= 0.0:
            DC[i] = 0.0
            continue
        zz = np.linspace(0.0, zi, int(n), dtype=float)
        Ez = E_ECTI(zz, theta)
        DC[i] = (c_light / H0) * float(_trap(1.0 / Ez, x=zz))

    return DC if np.ndim(z) else float(DC[0])

def D_M_ECTI(z, theta, n=4000):
    return D_C_ECTI(z, theta, n=n)

def D_L_ECTI(z, theta, n=4000):
    z = np.asarray(z, dtype=float)
    return (1.0 + z) * D_M_ECTI(z, theta, n=n)

def mu_ECTI(z, theta, n=4000):
    H0, Om0, _s8, _beta, _zt, M, _wb = _unpack_ecti_theta(theta)
    DL = D_L_ECTI(z, theta, n=n)
    return 5.0 * np.log10(DL) + 25.0 + M

def DH_ECTI(z, theta):
    return c_light / H_ECTI(z, theta)

def DV_ECTI(z, theta, n=4000):
    z = np.asarray(z, dtype=float)
    DM = D_M_ECTI(z, theta, n=n)
    DH = DH_ECTI(z, theta)
    return (DM**2 * (z * DH))**(1.0/3.0)

def _rs_from_theta(theta):
    H0, Om0, _s8, _beta, _zt, _M, wb = _unpack_ecti_theta(theta)
    return get_rs_star(H0, Om0, wb)

def DM_over_rs_ECTI(z, theta, n=4000):
    return D_M_ECTI(z, theta, n=n) / _rs_from_theta(theta)

def DH_over_rs_ECTI(z, theta):
    return DH_ECTI(z, theta) / _rs_from_theta(theta)

def DV_over_rs_ECTI(z, theta, n=4000):
    return DV_ECTI(z, theta, n=n) / _rs_from_theta(theta)

print("✔ ECTI background ready (FAIR k=5): E,H, distances, mu, BAO ratios, wb-compatible")

By construction, the ECTI-P background reduces exactly to ΛCDM when β = 0, independently of the choice of zₜ; this identity is explicitly verified in the proof cell below.

In [ ]:
# ============================================================
# PROOF CELL — ECTI reduces to ΛCDM when beta = 0 (zt arbitrary > 0)
#   Referee-proof identity checks:
#     (1) E(z) equality on grid
#     (2) Distance equality: D_M, D_L
#     (3) mu equality on z>0 (avoid log10(DL=0) at z=0)
#     (4) BAO: DV/rs equality
#   Output explicitly states that zt is arbitrary and irrelevant when beta=0.
# ============================================================

import numpy as np

_required = [
    "E_LCDM", "D_M_LCDM", "D_L_LCDM", "mu_LCDM", "DV_over_rs_LCDM",
    "E_ECTI", "D_M_ECTI", "D_L_ECTI", "mu_ECTI", "DV_over_rs_ECTI",
]
_missing = [k for k in _required if k not in globals()]
if _missing:
    raise NameError("Missing required symbols before PROOF cell: " + ", ".join(_missing))

# ------------------------------------------------------------
# Generic θ5 test point (must be within priors)
# θ5 = (H0, Om0, σ8, M, wb)
# ------------------------------------------------------------
theta5 = np.array([67.0, 0.315, 0.80, -19.40, 0.02237], dtype=float)
H0, Om0, s8, M, wb = theta5

# ------------------------------------------------------------
# Construct ECTI theta with beta=0 and arbitrary zt>0.
# Your ECTI unpacker supports θ7=(H0,Om0,s8,beta,zt,M,wb).
# Key identity: beta=0 => exp[beta*exp(-z/zt)] = 1 for all z, hence ΛCDM exactly.
# zt is only required to be >0 to satisfy the model guard; it is physically irrelevant when beta=0.
# ------------------------------------------------------------
beta0 = 0.0
zt_any = float(globals().get("ZT_FIXED", 0.10))
if zt_any <= 0.0:
    zt_any = 0.10  # purely to satisfy zt>0 guard; value irrelevant when beta=0

theta7_beta0 = np.array([H0, Om0, s8, beta0, zt_any, M, wb], dtype=float)

# ------------------------------------------------------------
# Numerical tolerances (referee-proof)
# ------------------------------------------------------------
rtol_E   = 3e-10
rtol_D   = 5e-9
atol_mu  = 5e-10   # absolute tolerance for mu equality (more stable than relative)
rtol_bao = 2e-8

# ------------------------------------------------------------
# z grid (include 0 for E and D; exclude 0 for mu to avoid log10(DL=0))
# ------------------------------------------------------------
z_grid = np.concatenate([np.linspace(0.0, 0.30, 120), np.linspace(0.30, 3.0, 180)])
z_mu   = z_grid[z_grid > 0.0]

# 1) E(z)
E_lcdm = np.asarray(E_LCDM(z_grid, H0, Om0), float)
E_ecti = np.asarray(E_ECTI(z_grid, theta7_beta0), float)
relE = float(np.max(np.abs(E_ecti - E_lcdm) / np.maximum(1e-15, np.abs(E_lcdm))))

# 2) distances
DM_l = np.asarray(D_M_LCDM(z_grid, H0, Om0), float)
DM_e = np.asarray(D_M_ECTI(z_grid, theta7_beta0), float)
relDM = float(np.max(np.abs(DM_e - DM_l) / np.maximum(1e-12, np.abs(DM_l))))

DL_l = np.asarray(D_L_LCDM(z_grid, H0, Om0), float)
DL_e = np.asarray(D_L_ECTI(z_grid, theta7_beta0), float)
relDL = float(np.max(np.abs(DL_e - DL_l) / np.maximum(1e-12, np.abs(DL_l))))

# 3) mu(z) on z>0 only
mu_l = np.asarray(mu_LCDM(z_mu, theta5), float)
mu_e = np.asarray(mu_ECTI(z_mu, theta7_beta0), float)
max_abs_dmu = float(np.max(np.abs(mu_e - mu_l)))

# 4) BAO DV/rs
if "BAO_z" in globals():
    z_bao = np.asarray(globals()["BAO_z"], float).ravel()
else:
    z_bao = np.array([0.15, 0.32, 0.51, 0.70, 1.48], dtype=float)

bao_l = np.asarray(DV_over_rs_LCDM(z_bao, theta5), float).ravel()
bao_e = np.asarray(DV_over_rs_ECTI(z_bao, theta7_beta0), float).ravel()
relbao = float(np.max(np.abs(bao_e - bao_l) / np.maximum(1e-12, np.abs(bao_l))))

# ------------------------------------------------------------
# Assertions (fail hard if identity breaks)
# ------------------------------------------------------------
if not np.allclose(E_ecti, E_lcdm, rtol=rtol_E, atol=0.0):
    raise RuntimeError(f"E(z) identity failed at beta=0. max_rel={relE:.3e}")
if not np.allclose(DM_e, DM_l, rtol=rtol_D, atol=0.0):
    raise RuntimeError(f"D_M identity failed at beta=0. max_rel={relDM:.3e}")
if not np.allclose(DL_e, DL_l, rtol=rtol_D, atol=0.0):
    raise RuntimeError(f"D_L identity failed at beta=0. max_rel={relDL:.3e}")
if not np.allclose(mu_e, mu_l, rtol=0.0, atol=atol_mu):
    raise RuntimeError(f"mu identity failed at beta=0. max_abs_dmu={max_abs_dmu:.3e}")
if not np.allclose(bao_e, bao_l, rtol=rtol_bao, atol=0.0):
    raise RuntimeError(f"DV/rs identity failed at beta=0. max_rel={relbao:.3e}")

print("====================================================")
print("PROOF: ECTI → ΛCDM when β = 0 (zt arbitrary > 0)")
print("----------------------------------------------------")
print("theta5 (ΛCDM)       =", theta5, "  (H0,Om0,σ8,M,wb)")
print(
    "theta7 (ECTI, β=0)  =", theta7_beta0,
    f"\n  with zt={zt_any:.3f} chosen arbitrarily (>0) — physically irrelevant when β=0"
)
print("----------------------------------------------------")
print(f"max_rel |E|     = {relE:.3e}   (rtol={rtol_E})")
print(f"max_rel |D_M|   = {relDM:.3e}  (rtol={rtol_D})")
print(f"max_rel |D_L|   = {relDL:.3e}  (rtol={rtol_D})")
print(f"max_abs |Δmu|   = {max_abs_dmu:.3e} (atol={atol_mu})  [z>0]")
print(f"max_rel |DV/rs| = {relbao:.3e} (rtol={rtol_bao})")
print("====================================================")
print("✔ Identity checks PASSED (numerical equality within tolerance).")

# 5) Likelihood blocks (χ² per probe)

This section defines the independent χ² contributions for each probe:
SN, BAO, RSD, KiDS, Planck compressed prior (R, ℓA, ωb), and Planck lensing-derived 1D constraint.


In [ ]:
# ============================================================
# 5.1) Supernovae likelihood (Pantheon+SH0ES: m_b_corr + full cov)
#    FAIR: θ5 = (H0, Om0, σ8, M, wb) for BOTH ΛCDM and ECTI
# ============================================================

import numpy as np

def chi2_SN_LCDM(theta):
    mu_th = mu_LCDM(z_SN, theta)
    diff = mu_SN - mu_th
    return float(diff @ Cinv_SN @ diff)

def chi2_SN_ECTI(theta):
    mu_th = mu_ECTI(z_SN, theta)
    diff = mu_SN - mu_th
    return float(diff @ Cinv_SN @ diff)

def lnlikelihood_sn_lcdm(theta):
    return -0.5 * chi2_SN_LCDM(theta)

def lnlikelihood_sn_ecti(theta):
    return -0.5 * chi2_SN_ECTI(theta)

print("✔ SN likelihood ready (ΛCDM/ECTI): chi2_SN_*, lnlikelihood_sn_*")


In [ ]:
# ============================================================
# 5.2) DESI DR1 BAO likelihood (mean + cov)
#    Coherent early-time: uses r_d(wb) from Cell 3bis (radiation + wb)
#    FAIR: θ5 = (H0, Om0, σ8, M, wb)
# ============================================================

import numpy as np

_required = ["z_BAO", "BAO_obs", "BAO_inv_cov", "BAO_row_kind",
             "D_M_LCDM", "H_LCDM",
             "D_M_ECTI", "DH_ECTI", "DV_ECTI",
             "rd_drag", "c_light"]
_missing = [n for n in _required if n not in globals()]
if _missing:
    raise NameError("Missing required symbols before BAO cell: " + ", ".join(_missing))

def _theta5(theta, label="theta"):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError(f"{label} must be θ5=(H0,Om0,σ8,M,wb); got size {th.size}")
    return th

def bao_theory_vector_LCDM(theta, n=4000):
    th = _theta5(theta, "BAO LCDM theta")
    H0, Om0, _s8, _M, wb = th
    rd = rd_drag(H0, Om0, wb)

    vec = np.zeros_like(BAO_obs, dtype=float)
    for i, (zi, kind) in enumerate(zip(z_BAO, BAO_row_kind)):
        zi = float(zi)
        if kind == "DM_over_rd":
            dm = D_M_LCDM(zi, float(H0), float(Om0), n=n)
            vec[i] = dm / rd
        elif kind == "DH_over_rd":
            dh = c_light / H_LCDM(zi, float(H0), float(Om0))
            vec[i] = dh / rd
        elif kind == "DV_over_rd":
            # DV = (DM^2 * z * DH)^(1/3)
            dm = D_M_LCDM(zi, float(H0), float(Om0), n=n)
            dh = c_light / H_LCDM(zi, float(H0), float(Om0))
            dv = (dm*dm * (zi*dh))**(1.0/3.0)
            vec[i] = dv / rd
        else:
            raise ValueError(f"Unknown BAO_row_kind: {kind}")
    return vec

def bao_theory_vector_ECTI(theta, n=4000):
    th = _theta5(theta, "BAO ECTI theta")
    H0, Om0, _s8, _M, wb = th
    rd = rd_drag(H0, Om0, wb)  # same early-time physics

    vec = np.zeros_like(BAO_obs, dtype=float)
    for i, (zi, kind) in enumerate(zip(z_BAO, BAO_row_kind)):
        zi = float(zi)
        if kind == "DM_over_rd":
            vec[i] = D_M_ECTI(zi, th, n=n) / rd
        elif kind == "DH_over_rd":
            vec[i] = DH_ECTI(zi, th) / rd
        elif kind == "DV_over_rd":
            vec[i] = DV_ECTI(zi, th, n=n) / rd
        else:
            raise ValueError(f"Unknown BAO_row_kind: {kind}")
    return vec

def chi2_BAO_LCDM(theta):
    t = bao_theory_vector_LCDM(theta)
    diff = BAO_obs - t
    return float(diff @ BAO_inv_cov @ diff)

def chi2_BAO_ECTI(theta):
    t = bao_theory_vector_ECTI(theta)
    diff = BAO_obs - t
    return float(diff @ BAO_inv_cov @ diff)

def lnlikelihood_bao_lcdm(theta):
    return -0.5 * chi2_BAO_LCDM(theta)

def lnlikelihood_bao_ecti(theta):
    return -0.5 * chi2_BAO_ECTI(theta)

print("✔ BAO likelihood ready (coherent r_d(wb) + radiation): chi2_BAO_*")


In [ ]:
# ============================================================
# 5.3) RSD likelihood — fσ₈(z) with linear growth ODE (σ8 FREE)
#    FAIR: θ5 = (H0, Om0, σ8, M, wb) for BOTH ΛCDM and ECTI
#    Growth uses the SAME background E(z) definitions as SN/BAO.
# ============================================================

import numpy as np
from scipy.integrate import solve_ivp

_required = ["z_RSD", "fs8_obs", "fs8_err"]
_missing = [n for n in _required if n not in globals()]
if _missing:
    raise NameError("Missing required RSD arrays before RSD cell: " + ", ".join(_missing))

def _check_theta5(theta, label):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError(f"{label} expects θ5=(H0,Om0,σ8,M,wb); got size {th.size}")
    return th

def _E_of_z(model, z, theta):
    if model == "LCDM":
        H0, Om0, *_ = _unpack_lcdm_theta(theta)
        return E_LCDM(z, H0, Om0)
    elif model == "ECTI":
        return E_ECTI(z, theta)
    else:
        raise ValueError("model must be 'LCDM' or 'ECTI'")

def _dlnH_dlnA(model, a, theta):
    # numerical derivative in ln a (robust, no hard-coded formula)
    # H(a) = H0 * E(z) with z=1/a-1
    eps = 1e-4
    lna = np.log(a)
    a1 = np.exp(lna - eps)
    a2 = np.exp(lna + eps)
    z1 = 1.0 / a1 - 1.0
    z2 = 1.0 / a2 - 1.0
    H1 = _E_of_z(model, z1, theta)
    H2 = _E_of_z(model, z2, theta)
    # H0 cancels in derivative
    return float((np.log(H2) - np.log(H1)) / (2.0 * eps))

def _growth_solution(model, theta, a_min=1e-3):
    """
    Solve for D(a) with initial conditions deep in matter era:
      D(a_min) = a_min
      dD/da(a_min) = 1
    Then normalize to D(1)=1.
    Returns interpolants via dense_output for D(a) and dD/da(a).
    """
    th = _check_theta5(theta, f"growth_solution[{model}]")
    H0, Om0, sigma8, _M, _wb = th

    def rhs(a, y):
        # y = [D, dD/da]
        D, dDda = y
        z = 1.0 / a - 1.0
        E = float(_E_of_z(model, z, th))
        if not np.isfinite(E) or E <= 0.0:
            return [np.nan, np.nan]

        dlnH_dlnA = _dlnH_dlnA(model, a, th)

        # Equation: D'' + [ (3/a) + (dlnH/dlna)/a ] D' - (3/2) Om0 /(a^5 E^2) D = 0
        # Note: H(a)^2 = H0^2 E^2; H0 cancels.
        coeff_fric = (3.0 + dlnH_dlnA) / a
        coeff_src  = 1.5 * Om0 / (a**5 * E**2)

        d2Dda2 = -coeff_fric * dDda + coeff_src * D
        return [dDda, d2Dda2]

    y0 = [a_min, 1.0]
    sol = solve_ivp(
        rhs, t_span=(a_min, 1.0), y0=y0, rtol=1e-6, atol=1e-9,
        dense_output=True, max_step=0.02
    )
    if not sol.success:
        raise RuntimeError(f"Growth ODE failed for {model}: {sol.message}")

    # Normalize to D(1)=1
    D1 = float(sol.y[0, -1])
    def D_of_a(a):
        return sol.sol(a)[0] / D1
    def dDda_of_a(a):
        return sol.sol(a)[1] / D1

    return D_of_a, dDda_of_a

def fsigma8_theory(theta, model="LCDM"):
    th = _check_theta5(theta, f"fsigma8_theory[{model}]")
    sigma8 = float(th[2])

    D_of_a, dDda_of_a = _growth_solution(model, th)

    fs8 = np.zeros_like(z_RSD, dtype=float)
    for i, z in enumerate(z_RSD):
        a = 1.0 / (1.0 + float(z))
        D  = float(D_of_a(a))
        dDda = float(dDda_of_a(a))
        if D <= 0.0 or not np.isfinite(D) or not np.isfinite(dDda):
            fs8[i] = np.nan
            continue
        f = (a / D) * dDda  # d ln D / d ln a
        fs8[i] = f * sigma8 * D
    return fs8

def chi2_RSD_LCDM(theta):
    pred = fsigma8_theory(theta, model="LCDM")
    diff = (fs8_obs - pred) / fs8_err
    return float(np.dot(diff, diff))

def chi2_RSD_ECTI(theta):
    pred = fsigma8_theory(theta, model="ECTI")
    diff = (fs8_obs - pred) / fs8_err
    return float(np.dot(diff, diff))

def lnlikelihood_rsd_lcdm(theta):
    return -0.5 * chi2_RSD_LCDM(theta)

def lnlikelihood_rsd_ecti(theta):
    return -0.5 * chi2_RSD_ECTI(theta)

print("✔ RSD likelihood ready (ΛCDM/ECTI): chi2_RSD_*, lnlikelihood_rsd_*")

## 5.4 KiDS (compressed weak-lensing constraint)

KiDS likelihood.  
We use a compressed KiDS constraint (effective S₈-like information), rather than the full weak-lensing two-point correlation function likelihood.  
This choice is consistent with the background-only scope of this work and avoids introducing additional perturbation-level modeling.


In [ ]:
# ============================================================
# 5.4) KiDS S8 prior likelihood (σ8 FREE)
#    FAIR: same S8 definition for ΛCDM and ECTI
# ============================================================

import numpy as np

def _S8_from_theta(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError("KiDS expects θ5=(H0,Om0,σ8,M,wb).")
    Om0 = float(th[1])
    s8  = float(th[2])
    return s8 * np.sqrt(Om0 / 0.3)

def chi2_KiDS_LCDM(theta):
    if not globals().get("USE_KIDS", False):
        return 0.0
    S8 = _S8_from_theta(theta)
    return float(((S8 - KIDS_MEAN) / KIDS_SIGMA) ** 2)

def chi2_KiDS_ECTI(theta):
    return chi2_KiDS_LCDM(theta)

def lnlikelihood_kids_lcdm(theta):
    return -0.5 * chi2_KiDS_LCDM(theta)

def lnlikelihood_kids_ecti(theta):
    return -0.5 * chi2_KiDS_ECTI(theta)

print("✔ KiDS prior ready (ΛCDM/ECTI): chi2_KiDS_*, lnlikelihood_kids_*")


In [ ]:
# ============================================================
# 5.5) CMB COMPRESSED LIKELIHOOD (PLANCK lite: R + ℓA + wb, 3×3)
#    Coherent early-time: D_M(z*) + r_s(z*) include radiation + wb
#    FAIR: θ5 = (H0, Om0, σ8, M, wb) for BOTH ΛCDM and ECTI
# ============================================================

import numpy as np

_required = ["D_M_early", "rs_star", "c_light", "Z_STAR"]
_missing = [n for n in _required if n not in globals()]
if _missing:
    raise NameError("Missing required symbols before CMB cell: " + ", ".join(_missing))

# Planck lite compressed observations [R, lA, wb]
CMB_OBS = np.array([1.7502, 301.471, 0.02237], dtype=float)

# Covariance (R, lA, wb)
CMB_COV = np.array([
    [2.125e-05, -1.650e-04, -1.000e-07],
    [-1.650e-04, 8.836e-03,  1.300e-06],
    [-1.000e-07, 1.300e-06,  2.250e-08]
], dtype=float)
CMB_INV_COV = np.linalg.inv(CMB_COV)

def _theta5(theta, label="theta"):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError(f"{label} must be θ5=(H0,Om0,σ8,M,wb); got size {th.size}")
    return th

def _cmb_vector_R_lA_wb(theta):
    th = _theta5(theta, "CMB theta")
    H0, Om0, _s8, _M, wb = th

    dm_star = D_M_early(float(Z_STAR), float(H0), float(Om0), float(wb))
    rs      = rs_star(float(H0), float(Om0), float(wb))

    lA = np.pi * dm_star / rs
    R  = np.sqrt(float(Om0)) * (float(H0) / float(c_light)) * dm_star
    return np.array([R, lA, wb], dtype=float)

def chi2_CMB_LCDM(theta):
    vec = _cmb_vector_R_lA_wb(theta)
    diff = vec - CMB_OBS
    return float(diff @ CMB_INV_COV @ diff)

def chi2_CMB_ECTI(theta):
    # FAIR: same early-time constraint, depends only on (H0, Om0, wb)
    return chi2_CMB_LCDM(theta)

def lnlikelihood_cmb_lcdm(theta):
    return -0.5 * chi2_CMB_LCDM(theta)

def lnlikelihood_cmb_ecti(theta):
    return -0.5 * chi2_CMB_ECTI(theta)

print("✔ CMB compressed ready (Planck lite 3D): radiation + r_s(z*) + wb coherent")

In [ ]:
# ============================================================
# 5.6) Planck lensing-derived 1D prior (summary constraint; lensing-only)
#   This is NOT the Planck lensing likelihood (no C_L^{phi-phi}, no covariance).
#   It is a published 1D derived constraint from Planck 2018 lensing-only analysis:
#
#     sigma8 * Omega_m^alpha = mean ± sigma   with alpha=0.25
#
#   Usage policy (referee-proof):
#     - chi2_PLANCK_LENSING(theta) ALWAYS returns the χ² value.
#     - The toggle USE_PLANCK_LENSING is applied ONLY in chi2_tot / chi2_breakdown.
#     - No new parameter (k unchanged).
# ============================================================

import numpy as np

# Toggle (applied in chi2_tot / chi2_breakdown; NOT inside chi2_PLANCK_LENSING)
USE_PLANCK_LENSING = bool(globals().get("USE_PLANCK_LENSING", True))

# --- Published lensing-only derived constraint (Planck 2018 lensing) ---
LENSING_S8OM025_ALPHA = float(globals().get("LENSING_S8OM025_ALPHA", 0.25))
LENSING_S8OM025_MEAN  = float(globals().get("LENSING_S8OM025_MEAN",  0.589))
LENSING_S8OM025_SIGMA = float(globals().get("LENSING_S8OM025_SIGMA", 0.020))

# Traceability (string for paper/README; keep it in the notebook globals)
LENSING_S8OM025_REF = str(globals().get(
    "LENSING_S8OM025_REF",
    "Planck Collaboration VIII (Planck 2018 lensing-only; A&A 641, A8 (2020); see text/eq. for sigma8*Omega_m^0.25 = 0.589±0.020)"
))

def lensing_combo_s8_om025(theta):
    """
    Returns: sigma8 * Om0^alpha from theta5 = (H0, Om0, sigma8, M, wb).
    """
    th = np.asarray(theta, dtype=float).ravel()
    if th.size < 5:
        raise ValueError(f"lensing_combo_s8_om025 expects θ5 (>=5); got {th.size}")

    Om0    = float(th[1])
    sigma8 = float(th[2])
    alpha  = float(LENSING_S8OM025_ALPHA)

    # Hard physical guards (avoid silent nonsense)
    if (not np.isfinite(Om0)) or (not np.isfinite(sigma8)):
        return np.nan
    if Om0 <= 0.0 or sigma8 <= 0.0:
        return np.nan

    return sigma8 * (Om0 ** alpha)

def chi2_PLANCK_LENSING(theta):
    """
    Independent 1D χ² block:
      χ² = ((combo - mean)/sigma)^2
    NOTE: toggle is NOT here (handled by chi2_tot / chi2_breakdown).
    """
    combo = float(lensing_combo_s8_om025(theta))
    if not np.isfinite(combo):
        return np.inf
    d = (combo - float(LENSING_S8OM025_MEAN)) / float(LENSING_S8OM025_SIGMA)
    return float(d * d)

print("✔ Planck lensing-derived 1D prior loaded (summary constraint)")
print(f"  USE_PLANCK_LENSING = {USE_PLANCK_LENSING}")
print(f"  combo = sigma8 * Om0^{LENSING_S8OM025_ALPHA:.2f}")
print(f"  mean ± sigma = {LENSING_S8OM025_MEAN:.3f} ± {LENSING_S8OM025_SIGMA:.3f}")
print(f"  ref = {LENSING_S8OM025_REF}")

At this stage, all independent χ² contributions entering the global likelihood
have been defined. The total likelihood is constructed as the sum of these blocks,
ensuring modularity, transparency, and full control over probe inclusion.


# 6) Priors, total χ², and log-probability (ΛCDM and ECTI-P)

This section defines:
- parameter priors (uniform bounds),
- the total χ² for each model as the sum of all enabled datasets,
- the log-probability function used by `emcee`.

The goal is to ensure a **single, explicit, auditable definition** of:
- what parameters are sampled,
- what priors are applied,
- which datasets contribute to the likelihood.

From this point onward, ΛCDM and ECTI-P are fully specified and ready for MCMC sampling.

In [ ]:
# ============================================================
# 6.1) Priors + total chi2 + lnprob (FAIR k=5, σ8 FREE, wb FREE)
#    θ5 = (H0, Om0, σ8, M, wb) for BOTH ΛCDM and ECTI
#
#    UPDATED:
#      - Adds Planck lensing derived 1D χ² block (no new params)
#      - Ensures chi2_tot includes LENS iff USE_PLANCK_LENSING=True
# ============================================================

import numpy as np

def lnprior_common(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        return -np.inf
    H0, Om0, sigma8, M, wb = th

    # --- Broad, explicit, auditable bounds (uniform) ---
    if not (40.0 < H0 < 95.0):
        return -np.inf
    if not (0.05 < Om0 < 0.6):
        return -np.inf
    if not (0.2 < sigma8 < 1.6):
        return -np.inf
    if not (-25.0 < M < -15.0):
        return -np.inf
    if not (0.005 < wb < 0.04):
        return -np.inf

    return 0.0

def lnprior_LCDM(theta):
    return lnprior_common(theta)

def lnprior_ECTI(theta):
    return lnprior_common(theta)

def _chi2_lensing_if_on(theta):
    if not bool(globals().get("USE_PLANCK_LENSING", False)):
        return 0.0
    if "chi2_PLANCK_LENSING" not in globals():
        raise NameError("USE_PLANCK_LENSING=True but chi2_PLANCK_LENSING is not defined (Cell 5.6 missing).")
    return float(globals()["chi2_PLANCK_LENSING"](theta))

def chi2_tot_LCDM(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        return np.inf

    chi2 = (
        float(chi2_SN_LCDM(th))
        + float(chi2_BAO_LCDM(th))
        + float(chi2_RSD_LCDM(th))
        + float(chi2_CMB_LCDM(th))
    )

    if bool(globals().get("USE_KIDS", False)):
        chi2 += float(chi2_KiDS_LCDM(th))

    chi2 += _chi2_lensing_if_on(th)

    return float(chi2)

def chi2_tot_ECTI(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        return np.inf

    chi2 = (
        float(chi2_SN_ECTI(th))
        + float(chi2_BAO_ECTI(th))
        + float(chi2_RSD_ECTI(th))
        + float(chi2_CMB_ECTI(th))
    )

    if bool(globals().get("USE_KIDS", False)):
        chi2 += float(chi2_KiDS_ECTI(th))

    chi2 += _chi2_lensing_if_on(th)

    return float(chi2)

def lnprob_LCDM(theta):
    lp = lnprior_LCDM(theta)
    if not np.isfinite(lp):
        return -np.inf
    chi2 = chi2_tot_LCDM(theta)
    if not np.isfinite(chi2):
        return -np.inf
    return lp - 0.5 * chi2

def lnprob_ECTI(theta):
    lp = lnprior_ECTI(theta)
    if not np.isfinite(lp):
        return -np.inf
    chi2 = chi2_tot_ECTI(theta)
    if not np.isfinite(chi2):
        return -np.inf
    return lp - 0.5 * chi2

print("✔ Priors + lnprob ready (FAIR k=5): lnprob_LCDM, lnprob_ECTI",
      f"| USE_PLANCK_LENSING={bool(globals().get('USE_PLANCK_LENSING', False))}")


# 7) Walker initialization and χ² diagnostics

This section defines:

• Stable initialization centers for ΛCDM and ECTI-P  
• Randomized walker starting points (p0)  
• χ² breakdown helpers used after each MCMC run  

This cell **does NOT launch any MCMC** and is safe under
"Restart runtime & Run all".

In [ ]:
# ============================================================
# 7.1) Walker initialization and χ² diagnostics (FAIR k=5)
#    θ5 = (H0, Om0, σ8, M, wb) for BOTH ΛCDM and ECTI
#
#    UPDATED:
#      - chi2_breakdown_* includes LENS block when USE_PLANCK_LENSING=True
#      - Keeps return signature: (parts_dict, total_float)
# ============================================================

import numpy as np

SIGMA8_INIT = 0.811
WB_INIT     = 0.02237

# Stable init centers
INIT_CENTER = np.array([67.4, 0.315, SIGMA8_INIT, -19.4, WB_INIT], dtype=float)

def p0_theta5(nwalkers, scale=None, rng=None):
    """
    Gaussian cloud around INIT_CENTER with explicit per-parameter scales.
    Returns p0 shape (nwalkers, 5).
    """
    if rng is None:
        rng = np.random.default_rng()

    if scale is None:
        # conservative, auditable widths (tune if needed)
        scale = np.array([2.0, 0.05, 0.06, 0.25, 0.002], dtype=float)

    scale = np.asarray(scale, dtype=float).ravel()
    if scale.size != 5:
        raise ValueError("scale must be length 5 for θ5.")

    p0 = INIT_CENTER[None, :] + rng.normal(size=(int(nwalkers), 5)) * scale[None, :]
    return np.asarray(p0, dtype=float)

def _chi2_lensing_if_on(theta):
    if not bool(globals().get("USE_PLANCK_LENSING", False)):
        return 0.0
    if "chi2_PLANCK_LENSING" not in globals():
        raise NameError("USE_PLANCK_LENSING=True but chi2_PLANCK_LENSING is not defined (Cell 5.6 missing).")
    return float(globals()["chi2_PLANCK_LENSING"](theta))

def chi2_breakdown_LCDM(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError("chi2_breakdown_LCDM expects θ5.")
    parts = {
        "SN":   float(chi2_SN_LCDM(th)),
        "BAO":  float(chi2_BAO_LCDM(th)),
        "RSD":  float(chi2_RSD_LCDM(th)),
        "KiDS": float(chi2_KiDS_LCDM(th)) if bool(globals().get("USE_KIDS", False)) else 0.0,
        "CMB":  float(chi2_CMB_LCDM(th)),
        "LENS": float(_chi2_lensing_if_on(th)) if bool(globals().get("USE_PLANCK_LENSING", False)) else 0.0,
    }
    return parts, float(sum(parts.values()))

def chi2_breakdown_ECTI(theta):
    th = np.asarray(theta, dtype=float).ravel()
    if th.size != 5:
        raise ValueError("chi2_breakdown_ECTI expects θ5.")
    parts = {
        "SN":   float(chi2_SN_ECTI(th)),
        "BAO":  float(chi2_BAO_ECTI(th)),
        "RSD":  float(chi2_RSD_ECTI(th)),
        "KiDS": float(chi2_KiDS_ECTI(th)) if bool(globals().get("USE_KIDS", False)) else 0.0,
        "CMB":  float(chi2_CMB_ECTI(th)),
        "LENS": float(_chi2_lensing_if_on(th)) if bool(globals().get("USE_PLANCK_LENSING", False)) else 0.0,
    }
    return parts, float(sum(parts.values()))

# Quick deterministic diagnostics at INIT_CENTER (does NOT require MCMC)
try:
    parts_L, tot_L = chi2_breakdown_LCDM(INIT_CENTER)
    parts_E, tot_E = chi2_breakdown_ECTI(INIT_CENTER)
    print("--- χ² breakdown @ INIT_CENTER (FAIR k=5) ---")
    print("ΛCDM :", parts_L, "Total=", tot_L)
    print("ECTI :", parts_E, "Total=", tot_E)
except Exception as _e:
    print("⚠ χ² breakdown diagnostic failed (check data availability):", repr(_e))

print("✔ 10bis ready: p0_theta5 + chi2_breakdown_* (FAIR k=5)",
      f"| USE_PLANCK_LENSING={bool(globals().get('USE_PLANCK_LENSING', False))}")


# 8) MCMC runner (multi-run, autosave, protected)

This section executes the full MCMC campaign for ΛCDM and ECTI-P.

Safety features:
• Controlled by RUN_MCMC flag
• One directory per campaign
• One file per independent run
• Autosave after each run
• Safe under "Run all" when RUN_MCMC = False

In [ ]:
# ============================================================
# 8.1) MCMC runner — FAIR k=5 (θ5 = H0, Om0, σ8, M, wb)
#      Planck compressed priors: (R + ℓA + ωb) everywhere
#      Lensing-derived 1D prior: optional via USE_PLANCK_LENSING (no new params)
#
#      ✅ ORDER: PAIRS (LCDM run i -> ECTI run i -> Δχ² + Δblocks)
#      ✅ LOGS : NO progress bar, print every PRINT_EVERY steps with ETA
#      ✅ IO   : HDF5 autosave + resume (safe resume with fallback)
# ============================================================

import os, time, json
import numpy as np
import emcee
from emcee.backends import HDFBackend
from datetime import datetime

# ---------------------------
# Required symbols (strict)
# ---------------------------
_required = [
    "RUN_MCMC", "N_WALKERS", "N_STEPS", "N_BURN",
    "lnprob_LCDM", "lnprob_ECTI",
    "lnprior_LCDM", "lnprior_ECTI",
    "chi2_tot_LCDM", "chi2_tot_ECTI",
    "chi2_breakdown_LCDM", "chi2_breakdown_ECTI",
]
_missing = [n for n in _required if n not in globals()]
if _missing:
    raise NameError("Missing required symbols before Cell 8.1: " + ", ".join(_missing))

# ---------------------------
# Config (single source of truth)
# ---------------------------
NDIM        = 5
N_WALKERS   = int(N_WALKERS)
N_STEPS     = int(N_STEPS)
N_BURN      = int(N_BURN)
THIN        = int(globals().get("THIN", 1))
MOVE_A      = float(globals().get("MOVE_A", 2.0))
PRINT_EVERY = int(globals().get("PRINT_EVERY", 1000))
N_RUNS      = int(globals().get("N_RUNS", 3))
USE_KIDS    = bool(globals().get("USE_KIDS", False))
USE_PLANCK_LENSING = bool(globals().get("USE_PLANCK_LENSING", False))

if N_WALKERS <= 2 * NDIM:
    raise ValueError(f"N_WALKERS must be > 2*NDIM (=10) for StretchMove. Got {N_WALKERS}.")
if N_STEPS <= N_BURN + 10:
    raise ValueError("N_STEPS must be > N_BURN + 10")
if THIN < 1:
    raise ValueError("THIN must be >= 1")
if N_RUNS < 1:
    raise ValueError("N_RUNS must be >= 1")
if PRINT_EVERY < 1:
    raise ValueError("PRINT_EVERY must be >= 1")

print("====================================================")
print("▶ MCMC runner (FAIR k=5) — PAIRED ORDER")
print(f"N_RUNS            = {N_RUNS}")
print(f"N_WALKERS         = {N_WALKERS}")
print(f"N_STEPS           = {N_STEPS}")
print(f"N_BURN            = {N_BURN}")
print(f"THIN              = {THIN}")
print(f"MOVE_A            = {MOVE_A}")
print(f"PRINT_EVERY       = {PRINT_EVERY}")
print(f"USE_KIDS          = {USE_KIDS}  (consumed inside chi2_* via globals())")
print(f"USE_PLANCK_LENSING= {USE_PLANCK_LENSING}")
print("====================================================")

if not RUN_MCMC:
    print("⏭ RUN_MCMC=False → skipping MCMC runs (safe Run All).")
else:
    # ---------------------------
    # Campaign directory (k5 naming)
    # ---------------------------
    CHAINS_ROOT = globals().get("CHAINS_ROOT", globals().get("CHAINS_DIR", "ECTI_runs/chains_sigma8free"))
    os.makedirs(CHAINS_ROOT, exist_ok=True)

    _CAMPAIGN_POINTER = os.path.join(CHAINS_ROOT, "LAST_CAMPAIGN_DIR_LCDM_vs_ECTI_k5.txt")

    CAMPAIGN_DIR = globals().get("CAMPAIGN_DIR", None)
    if (CAMPAIGN_DIR is None) or (not isinstance(CAMPAIGN_DIR, str)) or (len(CAMPAIGN_DIR.strip()) == 0) or (not os.path.isdir(CAMPAIGN_DIR)):
        if os.path.exists(_CAMPAIGN_POINTER):
            try:
                with open(_CAMPAIGN_POINTER, "r", encoding="utf-8") as f:
                    _cand = f.read().strip()
                if _cand and os.path.isdir(_cand):
                    CAMPAIGN_DIR = _cand
            except Exception:
                CAMPAIGN_DIR = None

    if (CAMPAIGN_DIR is None) or (not isinstance(CAMPAIGN_DIR, str)) or (len(CAMPAIGN_DIR.strip()) == 0) or (not os.path.isdir(CAMPAIGN_DIR)):
        CAMPAIGN_DIR = os.path.join(
            CHAINS_ROOT,
            f"campaign_LCDM_vs_ECTI_k5_{N_STEPS:05d}_{N_RUNS}pairs_FAIRk5_" + datetime.now().strftime("%Y%m%d_%H%M%S")
        )

    os.makedirs(CAMPAIGN_DIR, exist_ok=True)
    globals()["CAMPAIGN_DIR"] = CAMPAIGN_DIR

    try:
        with open(_CAMPAIGN_POINTER, "w", encoding="utf-8") as f:
            f.write(CAMPAIGN_DIR)
    except Exception:
        pass

    FIG_DIR  = os.path.join(CAMPAIGN_DIR, "figures")
    TAB_DIR  = os.path.join(CAMPAIGN_DIR, "tables")
    POST_DIR = os.path.join(CAMPAIGN_DIR, "posteriors")
    META_DIR = os.path.join(CAMPAIGN_DIR, "meta")
    os.makedirs(FIG_DIR, exist_ok=True)
    os.makedirs(TAB_DIR, exist_ok=True)
    os.makedirs(POST_DIR, exist_ok=True)
    os.makedirs(META_DIR, exist_ok=True)

    print("✔ CAMPAIGN_DIR =", CAMPAIGN_DIR)

    # ---------------------------
    # Save campaign config (trace + lensing constants)
    # ---------------------------
    _meta_cfg = dict(
        NDIM=NDIM, N_WALKERS=N_WALKERS, N_STEPS=N_STEPS, N_BURN=N_BURN,
        THIN=THIN, MOVE_A=MOVE_A, PRINT_EVERY=PRINT_EVERY,
        USE_KIDS=USE_KIDS, USE_PLANCK_LENSING=USE_PLANCK_LENSING,
        N_RUNS=N_RUNS,
        timestamp=datetime.now().isoformat(),
        order="PAIRED (LCDM_i then ECTI_i)",
        emcee_version=getattr(emcee, "__version__", "unknown"),
        LENSING=dict(
            USE=bool(USE_PLANCK_LENSING),
            alpha=float(globals().get("LENSING_S8OM025_ALPHA", np.nan)),
            mean=float(globals().get("LENSING_S8OM025_MEAN",  np.nan)),
            sigma=float(globals().get("LENSING_S8OM025_SIGMA", np.nan)),
            ref=str(globals().get("LENSING_S8OM025_REF", "")),
        ),
    )
    try:
        with open(os.path.join(META_DIR, "campaign_config.json"), "w", encoding="utf-8") as f:
            json.dump(_meta_cfg, f, indent=2)
    except Exception:
        pass

    # ---------------------------
    # Initialization helper (model-prior aware)
    # ---------------------------
    def init_walkers(center, lnprior_fn, scale=None, seed=42):
        rng = np.random.default_rng(int(seed))
        center = np.asarray(center, float).ravel()
        if center.size != NDIM:
            raise ValueError("center must be theta5")

        if scale is None:
            scale = np.array([0.35, 0.015, 0.040, 0.030, 0.0006], dtype=float)

        p0 = center[None, :] + rng.normal(size=(N_WALKERS, NDIM)) * scale[None, :]

        eps = 1e-10
        p0[:, 0] = np.clip(p0[:, 0], 40.0 + eps, 95.0 - eps)
        p0[:, 1] = np.clip(p0[:, 1], 0.05 + eps, 0.60 - eps)
        p0[:, 2] = np.clip(p0[:, 2], 0.30 + eps, 1.50 - eps)
        p0[:, 3] = np.clip(p0[:, 3], -21.5 + eps, -18.0 - eps)
        p0[:, 4] = np.clip(p0[:, 4], 0.005 + eps, 0.040 - eps)

        for i in range(N_WALKERS):
            tries = 0
            while (not np.isfinite(lnprior_fn(p0[i]))) and tries < 8000:
                p = center + rng.normal(size=NDIM) * scale
                p[0] = np.clip(p[0], 40.0 + eps, 95.0 - eps)
                p[1] = np.clip(p[1], 0.05 + eps, 0.60 - eps)
                p[2] = np.clip(p[2], 0.30 + eps, 1.50 - eps)
                p[3] = np.clip(p[3], -21.5 + eps, -18.0 - eps)
                p[4] = np.clip(p[4], 0.005 + eps, 0.040 - eps)
                p0[i] = p
                tries += 1
            if tries >= 8000:
                raise RuntimeError("Failed to init walkers with finite prior (per-model).")
        return p0

    # ---------------------------
    # Backend open/reset helper
    # ---------------------------
    def open_or_reset_backend(h5_path):
        backend = HDFBackend(h5_path)

        if not os.path.exists(h5_path):
            backend.reset(N_WALKERS, NDIM)
            return backend, 0, "NEW"

        try:
            it = int(backend.iteration)
        except Exception:
            backend.reset(N_WALKERS, NDIM)
            return backend, 0, "RESET_CORRUPT"

        if it > 0:
            try:
                chain0 = backend.get_chain(discard=0, thin=1, flat=False)
                if chain0.shape[1] != N_WALKERS or chain0.shape[2] != NDIM:
                    backend.reset(N_WALKERS, NDIM)
                    return backend, 0, "RESET_SHAPE"
            except Exception:
                backend.reset(N_WALKERS, NDIM)
                return backend, 0, "RESET_READFAIL"

        return backend, it, "RESUME" if it > 0 else "EMPTY"

    # ---------------------------
    # ETA helper
    # ---------------------------
    def _fmt_eta(seconds):
        seconds = float(seconds)
        if (not np.isfinite(seconds)) or seconds < 0:
            return "?"
        m = int(seconds // 60)
        h = m // 60
        m = m % 60
        return f"{h}h{m:02d}m" if h > 0 else f"{m}m"

    # ---------------------------
    # Run one run
    # ---------------------------
    def run_one(model_tag, rid, lnprob_fn, lnprior_fn, chi2_tot_fn, chi2_breakdown_fn, seed_base):
        h5_path = os.path.join(CAMPAIGN_DIR, f"{model_tag}_run{rid:03d}.h5")
        backend, it0, mode = open_or_reset_backend(h5_path)

        # resume-safe init_state
        if it0 > 0:
            print(f"\n▶ {model_tag} run{rid:03d}: RESUMING backend @ iter={it0} ({mode})")
            try:
                init_state = backend.get_last_sample()  # emcee State
            except Exception:
                # ultra-robust fallback: last positions
                try:
                    p_last = backend.get_chain(discard=0, thin=1, flat=False)[-1, :, :]
                    init_state = np.array(p_last, float)
                    print("  (fallback) resumed from last chain positions (no State).")
                except Exception as e:
                    raise RuntimeError(f"{model_tag} run{rid:03d}: cannot resume (fallback failed): {repr(e)}")
        else:
            center = np.asarray(globals().get("INIT_CENTER", [67.4, 0.315, 0.811, -19.4, 0.02237]), float).ravel()
            if center.size != NDIM:
                raise ValueError("INIT_CENTER must be θ5 (size 5) for this runner.")
            seed_used = int(seed_base + rid)
            print(f"\n▶ {model_tag} run{rid:03d}: NEW init ({mode}) | seed={seed_used}")
            p0 = init_walkers(center, lnprior_fn=lnprior_fn, seed=seed_used)
            init_state = p0

            try:
                with open(os.path.join(META_DIR, f"{model_tag}_run{rid:03d}_meta.json"), "w", encoding="utf-8") as f:
                    json.dump(dict(model=model_tag, run_id=rid, seed=seed_used, init_center=list(map(float, center))), f, indent=2)
            except Exception:
                pass

        sampler = emcee.EnsembleSampler(
            N_WALKERS, NDIM, lnprob_fn,
            backend=backend,
            moves=emcee.moves.StretchMove(a=MOVE_A)
        )

        it_start = int(backend.iteration)
        n_remaining = N_STEPS - it_start

        if n_remaining <= 0:
            print(f"⏭ {model_tag} run{rid:03d}: already at N_STEPS={N_STEPS}")
        else:
            print(f"▶ {model_tag} run{rid:03d}: running {n_remaining} steps (target N_STEPS={N_STEPS})")
            t0 = time.time()
            for _ in sampler.sample(init_state, iterations=n_remaining, progress=False, store=True):
                it_abs = it_start + int(sampler.iteration)
                if (it_abs % PRINT_EVERY) == 0 or it_abs == N_STEPS:
                    dt = time.time() - t0
                    done = max(1, it_abs - it_start)
                    rate = done / max(1e-12, dt)
                    eta_s = (N_STEPS - it_abs) / max(1e-12, rate)
                    print(f"  iter={it_abs}/{N_STEPS} | elapsed={dt/60:.1f} min | eta={_fmt_eta(eta_s)}")

        it_final = int(backend.iteration)
        if it_final <= N_BURN:
            raise RuntimeError(
                f"{model_tag} run{rid:03d}: backend.iteration={it_final} <= N_BURN={N_BURN}. "
                "Run not advanced enough to extract post-burn MAP."
            )

        chain = backend.get_chain(discard=N_BURN, thin=THIN, flat=False)
        logp  = backend.get_log_prob(discard=N_BURN, thin=THIN, flat=False)

        if chain.size == 0 or logp.size == 0:
            raise RuntimeError(f"{model_tag} run{rid:03d}: empty chain/logp after discard (check N_BURN/THIN).")

        ij = np.unravel_index(int(np.nanargmax(logp)), logp.shape)
        th_map = np.array(chain[ij[0], ij[1], :], float)

        parts, chi2_map_from_parts = chi2_breakdown_fn(th_map)
        chi2_map = float(chi2_tot_fn(th_map))

        if abs(float(chi2_map) - float(chi2_map_from_parts)) > 1e-8:
            raise RuntimeError(
                f"{model_tag} run{rid:03d}: mismatch chi2_tot(MAP)={chi2_map} vs sum(parts)={chi2_map_from_parts}"
            )

        if USE_PLANCK_LENSING and ("LENS" not in parts):
            print(f"⚠ WARNING: USE_PLANCK_LENSING=True but 'LENS' key not found in parts for {model_tag} run{rid:03d}.")

        print("\n====================================================")
        print(f"RESULT — {model_tag} run{rid:03d} (post-burn MAP)")
        print("theta_MAP =", np.array2string(th_map, precision=10))
        print(f"chi2_MAP  = {chi2_map:.6f}")
        print("parts     =", parts)
        print("====================================================\n")

        return th_map, float(chi2_map), dict(parts)

    # ---------------------------
    # Execute in PAIRS
    # ---------------------------
    SEED_BASE_LCDM = int(globals().get("SEED_BASE_LCDM", 20260110))
    SEED_BASE_ECTI = int(globals().get("SEED_BASE_ECTI", 20260111))

    pairs_summary = []
    run_rows = []

    def _delta_blocks(partsE, partsL):
        keys = ["SN","BAO","RSD","KiDS","CMB","LENS","TOTAL"]
        return {k: float(partsE.get(k, 0.0)) - float(partsL.get(k, 0.0)) for k in keys}

    for rid in range(1, N_RUNS + 1):
        print("====================================================")
        print(f"▶ PAIR {rid}/{N_RUNS}")
        print("====================================================")

        thL, chiL, partsL = run_one(
            "LCDM_k5", rid,
            lnprob_fn=lnprob_LCDM, lnprior_fn=lnprior_LCDM,
            chi2_tot_fn=chi2_tot_LCDM, chi2_breakdown_fn=chi2_breakdown_LCDM,
            seed_base=SEED_BASE_LCDM
        )
        thE, chiE, partsE = run_one(
            "ECTI_k5", rid,
            lnprob_fn=lnprob_ECTI, lnprior_fn=lnprior_ECTI,
            chi2_tot_fn=chi2_tot_ECTI, chi2_breakdown_fn=chi2_breakdown_ECTI,
            seed_base=SEED_BASE_ECTI
        )

        dchi = float(chiE - chiL)
        dblk = _delta_blocks(partsE, partsL)

        dAIC = dchi
        dBIC = dchi

        print("----------------------------------------------------")
        print(f"PAIR {rid:03d} SUMMARY (MAP)")
        print(f"LCDM chi2_MAP = {chiL:.6f}")
        print(f"ECTI chi2_MAP = {chiE:.6f}")
        print(f"Δχ²_MAP (ECTI−LCDM) = {dchi:+.6f}  (ΔAIC=ΔBIC=Δχ² for same-k)")
        print("Δχ² blocks (ECTI−LCDM):")
        for k in ["SN","BAO","RSD","KiDS","CMB","LENS","TOTAL"]:
            print(f"  Δ{k:5s} = {dblk.get(k, 0.0):+.6f}")
        print("----------------------------------------------------\n")

        pairs_summary.append(dict(
            rid=int(rid),
            LCDM=dict(theta_MAP=thL.tolist(), chi2_MAP=float(chiL), parts=partsL),
            ECTI=dict(theta_MAP=thE.tolist(), chi2_MAP=float(chiE), parts=partsE),
            dchi2_MAP=float(dchi),
            dAIC=float(dAIC),
            dBIC=float(dBIC),
            dchi2_blocks=dblk,
        ))

        run_rows.append((rid, chiL, chiE, dchi, dAIC, dBIC))

        try:
            with open(os.path.join(META_DIR, "pairs_summary.json"), "w", encoding="utf-8") as f:
                json.dump(dict(pairs=pairs_summary), f, indent=2)
        except Exception:
            pass

    print("====================================================")
    print("FINAL SUMMARY — FAIR k=5 (PAIRED runs)")
    print("----------------------------------------------------")
    print(" run_id   chi2_LCDM     chi2_ECTI       dchi2       dAIC       dBIC")
    for (rid, chiL, chiE, dchi, dAIC, dBIC) in run_rows:
        print(f"{rid:6d} {chiL:11.6f} {chiE:12.6f} {dchi:11.6f} {dAIC:11.6f} {dBIC:11.6f}")
    print("----------------------------------------------------")
    rid_ref, chiL_ref, chiE_ref, dchi_ref, *_ = run_rows[-1]
    print(f"REFERENCE = LAST common run_id = run{rid_ref:03d}")
    print(f"Δχ²_ref (ECTI−LCDM) = {dchi_ref:+.6f}")
    print("====================================================")

    globals()["PAIRS_SUMMARY_K5"] = pairs_summary
    print("✔ Stored: PAIRS_SUMMARY_K5")
    print("✔ MCMC campaign completed:", CAMPAIGN_DIR)


# 9) Convergence diagnostics — Campaign (R̂, τ_int, ESS)

This section performs **global convergence diagnostics** for the full MCMC campaign,
using the **automatically saved HDF5 backends** (`LCDM_run*.h5`, `ECTI_run*.h5`).

### Goals

For each model (ΛCDM and ECTI-P), this step is designed to:

- Assess **inter-chain convergence** using the **Gelman–Rubin statistic (R̂)**
- Estimate the **integrated autocorrelation time** τ_int for each parameter
- Compute the **Effective Sample Size (ESS)** as a measure of statistical robustness
- Ensure convergence is not an artifact of a single run

This analysis is **mandatory before any final interpretation or publication**.

---

### Methodology

- Each run (`run1`, `run2`, `run3`) is treated as an **independent MCMC chain**
- A **global burn-in (`N_BURN`)** is removed prior to diagnostics
- Chains are read **directly from the HDF5 backends**
- R̂ is computed **across independent runs**, not across walkers
- τ_int and ESS are estimated **per parameter**, then aggregated across runs

---

### Interpretation criteria

**Gelman–Rubin (R̂)**  
- R̂ < 1.01 → excellent convergence  
- R̂ < 1.05 → acceptable convergence  
- R̂ ≥ 1.05 → not converged (longer runs required)

**Effective Sample Size (ESS)**  
- ESS > 300 → minimum acceptable  
- ESS > 1000 → robust (publication-level)

**Autocorrelation time (τ_int)**  
- N_steps / τ_int ≫ 50 recommended  
- Large τ_int indicates strongly correlated chains

---

### Important notes

- This section is **fully HPC-compatible** (CPU, MPI, Slurm, cloud environments)
- No likelihood recomputation is performed
- Failure to estimate τ_int usually indicates insufficient chain length
- This step **does not replace posterior analysis**, it validates it

---

### Expected outcome

At the end of this section:

- The campaign convergence status is clearly established
- The adequacy of the chosen number of steps (e.g. 10,000) is quantified
- Posterior results can be interpreted **with statistical confidence**

➡️ Subsequent cells (10-13) can then be analyzed safely.

In [ ]:
# ============================================================
# 9.1) Convergence diagnostics (Rhat / tau_int / ESS) — FAIR k=5
#   - marche sur familles LCDM_k5 / ECTI_k5 dans CAMPAIGN_DIR
#   - calcule split-Rhat (walkers = chains)
#   - tau_int via emcee.autocorr.integrated_time (si possible)
#   - export CSV + JSON
# ============================================================

import os, re, json, time
import numpy as np
import pandas as pd
from emcee.backends import HDFBackend

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get('RUN_MCMC', True))
_cd = globals().get('CAMPAIGN_DIR', None)
_SKIP_POST = (not _RUN_MCMC) or (not isinstance(_cd, str)) or (not os.path.isdir(_cd))
if _SKIP_POST:
    print('⏭ Skipping Cell 9.1 (Convergence diagnostics): RUN_MCMC=False or CAMPAIGN_DIR missing/invalid.')
else:
    if not any(f.lower().endswith('.h5') for f in os.listdir(_cd)):
        _SKIP_POST = True
        print('⏭ Skipping Cell 9.1 (Convergence diagnostics): no .h5 files found in CAMPAIGN_DIR=' + str(_cd))

if not _SKIP_POST:

    # Rehydrate TAB_DIR if needed (Run-All safe)
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(_cd, "tables"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    # Rehydrate THIN if needed (safe default)
    if "THIN" not in globals():
        globals()["THIN"] = 1

    _required = ["CAMPAIGN_DIR", "N_BURN", "THIN", "TAB_DIR"]
    _missing = [k for k in _required if k not in globals()]
    if _missing:
        raise NameError("Missing required symbols before Cell 9.1: " + ", ".join(_missing))

    def _safe_backend(h5_path, tries=10, wait_s=0.6):
        last = None
        for _ in range(tries):
            try:
                return HDFBackend(h5_path)
            except OSError as e:
                last = e
                time.sleep(wait_s)
        raise last

    def _extract_runid(path):
        m = re.search(r"_run(\d+)\.h5$", os.path.basename(path), flags=re.IGNORECASE)
        return int(m.group(1)) if m else None

    def _list_family_files(campaign_dir):
        files = [f for f in os.listdir(campaign_dir) if f.lower().endswith(".h5")]
        pat = re.compile(r"^(.*)_run(\d+)\.h5$", re.IGNORECASE)
        buckets = {}
        for f in files:
            m = pat.match(f)
            if not m:
                continue
            fam = m.group(1).upper()
            rid = int(m.group(2))
            if not (fam.startswith("LCDM") or fam.startswith("ECTI")):
                continue
            buckets.setdefault(fam, []).append((rid, os.path.join(campaign_dir, f)))
        for fam in list(buckets.keys()):
            buckets[fam] = [p for _, p in sorted(buckets[fam], key=lambda x: x[0])]
        return buckets

    def _choose_family(buckets, kind="LCDM"):
        keys = sorted([k for k in buckets.keys() if k.startswith(kind.upper())])
        if not keys:
            return None
        if kind.upper() == "LCDM":
            for pref in ["LCDM_K5"]:
                for k in keys:
                    if pref in k:
                        return k
            return keys[0]
        else:
            for pref in ["ECTI_K5"]:
                for k in keys:
                    if pref in k:
                        return k
            return keys[0]

    def _split_rhat(x):
        x = np.asarray(x, dtype=float)
        if x.ndim != 2:
            raise ValueError("_split_rhat expects 2D (nsteps, nchains)")
        n, m = x.shape
        if n < 20 or m < 2:
            return np.nan
        n2 = n // 2
        if n2 < 10:
            return np.nan
        x = x[:2*n2, :]
        first = x[:n2, :]
        second = x[n2:2*n2, :]
        y = np.vstack([first.T, second.T])  # (2m, n2)
        M = y.shape[0]
        N = y.shape[1]
        chain_means = np.mean(y, axis=1)
        chain_vars  = np.var(y, axis=1, ddof=1)
        W = np.mean(chain_vars)
        B = N * np.var(chain_means, ddof=1)
        if not np.isfinite(W) or not np.isfinite(B) or W <= 0:
            return np.nan
        var_hat = (N - 1)/N * W + (1/N) * B
        return float(np.sqrt(var_hat / W))

    def _tau_int_emcee(y_1d):
        try:
            from emcee.autocorr import integrated_time
            t = integrated_time(np.asarray(y_1d, float), tol=50, quiet=True)
            t = np.atleast_1d(t).astype(float)
            return float(t[0]) if np.isfinite(t[0]) else np.nan
        except Exception:
            return np.nan

    def _diagnose_file(h5_path, burn, thin=1):
        b = _safe_backend(h5_path)
        it = int(getattr(b, "iteration", b.get_chain().shape[0]))
        burn_eff = int(min(burn, max(it - 1, 0)))
        chain = b.get_chain(discard=burn_eff, thin=int(thin), flat=False)
        if chain.shape[0] < 2:
            raise RuntimeError(f"Too short post-burn chain in {os.path.basename(h5_path)}")
        nstep, nwalk, ndim = chain.shape

        rhats = []
        taus  = []
        ess   = []
        for j in range(ndim):
            x = chain[:, :, j]  # (nstep, nwalk)
            rhat = _split_rhat(x)
            rhats.append(rhat)

            series = np.mean(x, axis=1)
            tau = _tau_int_emcee(series)
            taus.append(tau)

            if np.isfinite(tau) and tau > 0:
                ess_j = (nstep * nwalk) / tau
            else:
                ess_j = np.nan
            ess.append(float(ess_j))

        return dict(
            file=os.path.basename(h5_path),
            path=h5_path,
            iteration=it,
            burn=burn_eff,
            post_steps=int(nstep),
            nwalkers=int(nwalk),
            ndim=int(ndim),
            rhat_max=float(np.nanmax(rhats)),
            rhat_vec=[float(x) if np.isfinite(x) else None for x in rhats],
            tau_med=float(np.nanmedian(taus)),
            tau_vec=[float(x) if np.isfinite(x) else None for x in taus],
            ess_med=float(np.nanmedian(ess)),
            ess_vec=[float(x) if np.isfinite(x) else None for x in ess],
        )

    buckets = _list_family_files(CAMPAIGN_DIR)
    famL = _choose_family(buckets, "LCDM")
    famE = _choose_family(buckets, "ECTI")
    if famL is None or famE is None:
        raise RuntimeError(f"Families not found in CAMPAIGN_DIR. Found={sorted(buckets.keys())}")

    files = [(famL, p) for p in buckets[famL]] + [(famE, p) for p in buckets[famE]]

    rows = []
    meta = {"campaign_dir": CAMPAIGN_DIR, "families": {"LCDM": famL, "ECTI": famE}, "results": []}

    print("====================================================")
    print("9.1) Convergence diagnostics — FAIR k=5")
    print("----------------------------------------------------")
    print("CAMPAIGN_DIR =", CAMPAIGN_DIR)
    print("LCDM family  =", famL)
    print("ECTI family  =", famE)
    print("burn/thin    =", int(N_BURN), "/", int(THIN))
    print("====================================================")

    for fam, path in files:
        rid = _extract_runid(path)
        d = _diagnose_file(path, burn=int(N_BURN), thin=int(THIN))
        d["family"] = fam
        d["run_id"] = rid
        rows.append({
            "family": fam,
            "run_id": rid,
            "file": d["file"],
            "iteration": d["iteration"],
            "burn": d["burn"],
            "post_steps": d["post_steps"],
            "nwalkers": d["nwalkers"],
            "ndim": d["ndim"],
            "rhat_max": d["rhat_max"],
            "tau_med": d["tau_med"],
            "ess_med": d["ess_med"],
        })
        meta["results"].append(d)

    df = pd.DataFrame(rows).sort_values(["family", "run_id"]).reset_index(drop=True)
    print(df.to_string(index=False))

    csv_path = os.path.join(TAB_DIR, "convergence_diagnostics_k5.csv")
    json_path = os.path.join(TAB_DIR, "convergence_diagnostics_k5.json")
    df.to_csv(csv_path, index=False)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2, ensure_ascii=False)

    globals()["CONV_DF_9.1"] = df
    globals()["CONV_META_9.1"] = meta

    print("----------------------------------------------------")
    print("✔ Saved:", csv_path)
    print("✔ Saved:", json_path)
    print("✔ Stored: CONV_DF_9.1, CONV_META_9.1")
    print("====================================================")


## Limitations and scope

This notebook implements a late-time phenomenological extension (ECTI-P) and compares it to $\Lambda$CDM under a FAIR $k=5$ parameterization.

- CMB information is included via compressed distance priors ($R$, $\ell_A$, $\omega_b$), not the full CMB power-spectrum likelihood.
- The analysis does not model non-linear structure formation, baryonic feedback, or detailed growth systematics beyond the implemented RSD ($f\sigma_8$) and a KiDS $S_8$ prior.
- The goal is not to claim a replacement of the standard model, but to provide a clean, reproducible baseline to assess whether deeper tests (full CMB likelihood, extended modelling) are warranted.

# 10) Final results — χ², Δχ², AIC, BIC (publication summary)

This section summarizes the **final statistical comparison**
between ΛCDM and ECTI-P after convergence has been validated.

For each model we report:
• Best-fit χ² (total)
• Δχ² relative to ΛCDM
• AIC and ΔAIC
• BIC and ΔBIC

Definitions:
AIC  = χ² + 2k  
BIC  = χ² + k ln(N)

where:
k = number of free parameters  
N = total number of data points

This table is the **core quantitative result** of the paper.

In [ ]:
# ============================================================
# 10.1) Final results (MAP/χ²/AIC/BIC) — FAIR k=5 (LAST RUN POLICY)
#   - détecte familles LCDM_k5 / ECTI_k5 dans CAMPAIGN_DIR
#   - extrait MAP par run + choisit le LAST run (run_id max commun) comme référence
#   - calcule Δχ², ΔAIC, ΔBIC (k identique: k=5 vs k=5)
#   - REFEREE-PROOF:
#       * N_DATA inclut LENS=1 si USE_PLANCK_LENSING=True (cohérence BIC vs χ²_total)
#       * meta exporte la policy + toggles
#   - exporte: FINAL_DF_10.1, BEST_10.1, N_DATA_10.1, K_PARAMS_10.1
# ============================================================

import os, re, time, json
import numpy as np
import pandas as pd
from emcee.backends import HDFBackend

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get('RUN_MCMC', True))
_cd = globals().get('CAMPAIGN_DIR', None)
_SKIP_POST = (not _RUN_MCMC) or (not isinstance(_cd, str)) or (not os.path.isdir(_cd))
if _SKIP_POST:
    print('⏭ Skipping Cell 10.1 (Final results): RUN_MCMC=False or CAMPAIGN_DIR missing/invalid.')
else:
    if not any(f.lower().endswith('.h5') for f in os.listdir(_cd)):
        _SKIP_POST = True
        print('⏭ Skipping Cell 10.1 (Final results): no .h5 files found in CAMPAIGN_DIR=' + str(_cd))

if not _SKIP_POST:

    # Rehydrate output dirs (Run-All safe)
    TAB_DIR_LOCAL  = globals().get("TAB_DIR",  os.path.join(_cd, "tables"))
    FIG_DIR_LOCAL  = globals().get("FIG_DIR",  os.path.join(_cd, "figures"))
    POST_DIR_LOCAL = globals().get("POST_DIR", os.path.join(_cd, "posteriors"))
    META_DIR_LOCAL = globals().get("META_DIR", os.path.join(_cd, "meta"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    os.makedirs(FIG_DIR_LOCAL, exist_ok=True)
    os.makedirs(POST_DIR_LOCAL, exist_ok=True)
    os.makedirs(META_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"]  = TAB_DIR_LOCAL
    globals()["FIG_DIR"]  = FIG_DIR_LOCAL
    globals()["POST_DIR"] = POST_DIR_LOCAL
    globals()["META_DIR"] = META_DIR_LOCAL

    _required = [
        "CAMPAIGN_DIR", "N_BURN",
        "chi2_breakdown_LCDM", "chi2_breakdown_ECTI",
    ]
    _missing = [k for k in _required if k not in globals()]
    if _missing:
        raise NameError("Missing required symbols before Cell 10.1: " + ", ".join(_missing))

    # Prefer explicit N_DATA from data-loading cells if available
    N_SN   = int(globals().get("N_SN", 1701))
    N_BAO  = int(globals().get("N_BAO", 12))
    N_RSD  = int(globals().get("N_RSD", 7))
    N_KIDS = 1 if bool(globals().get("USE_KIDS", False)) else 0
    N_CMB  = 3
    N_LENS = 1 if bool(globals().get("USE_PLANCK_LENSING", False)) else 0
    N_DATA = int(N_SN + N_BAO + N_RSD + N_KIDS + N_CMB + N_LENS)

    K_PARAMS = 5  # FAIR k=5 in this notebook

    globals()["N_DATA_10.1"] = N_DATA
    globals()["K_PARAMS_10.1"] = K_PARAMS

    def _extract_runid(path):
        m = re.search(r"_run(\d+)\.h5$", os.path.basename(path), flags=re.IGNORECASE)
        return int(m.group(1)) if m else None

    def _list_family_files(campaign_dir):
        files = [f for f in os.listdir(campaign_dir) if f.lower().endswith(".h5")]
        pat = re.compile(r"^(.*)_run(\d+)\.h5$", re.IGNORECASE)
        buckets = {}
        for f in files:
            m = pat.match(f)
            if not m:
                continue
            fam = m.group(1).upper()
            rid = int(m.group(2))
            if not (fam.startswith("LCDM") or fam.startswith("ECTI")):
                continue
            buckets.setdefault(fam, []).append((rid, os.path.join(campaign_dir, f)))
        for fam in list(buckets.keys()):
            buckets[fam] = [p for _, p in sorted(buckets[fam], key=lambda x: x[0])]
        return buckets

    def _choose_family(buckets, kind="LCDM"):
        keys = sorted([k for k in buckets.keys() if k.startswith(kind.upper())])
        if not keys:
            return None
        pref = f"{kind.upper()}_K5"
        for k in keys:
            if pref in k:
                return k
        return keys[0]

    def _map_from_h5(h5_path, burn=0, thin=1):
        b = HDFBackend(h5_path)
        it = int(getattr(b, "iteration", b.get_chain().shape[0]))
        burn_eff = int(min(int(burn), max(it - 1, 0)))
        if it <= burn_eff:
            raise RuntimeError(f"H5 iteration={it} <= burn_eff={burn_eff} (post-burn empty): {os.path.basename(h5_path)}")
        chain = b.get_chain(discard=burn_eff, thin=int(thin), flat=False)
        logp  = b.get_log_prob(discard=burn_eff, thin=int(thin), flat=False)
        if chain.shape[0] < 1:
            raise RuntimeError(f"Empty post-burn chain in {os.path.basename(h5_path)}")
        ij = np.unravel_index(int(np.nanargmax(logp)), logp.shape)
        theta = np.array(chain[ij[0], ij[1], :], dtype=float).ravel()
        lp = float(logp[ij])
        where = (int(burn_eff + ij[0]), int(ij[1]))
        return theta, lp, where, it

    buckets = _list_family_files(CAMPAIGN_DIR)
    famL = _choose_family(buckets, "LCDM")
    famE = _choose_family(buckets, "ECTI")
    if famL is None or famE is None:
        raise RuntimeError(f"Families not found in CAMPAIGN_DIR. Found={sorted(buckets.keys())}")

    filesL = buckets[famL]
    filesE = buckets[famE]
    runidsL = [(_extract_runid(p), p) for p in filesL]
    runidsE = [(_extract_runid(p), p) for p in filesE]
    runidsL = [(rid, p) for rid, p in runidsL if rid is not None]
    runidsE = [(rid, p) for rid, p in runidsE if rid is not None]
    if not runidsL or not runidsE:
        raise RuntimeError("Could not parse run IDs from H5 files.")

    common = sorted(set(r for r,_ in runidsL) & set(r for r,_ in runidsE))
    if not common:
        raise RuntimeError("No common run_id between LCDM and ECTI families.")
    rid_ref = int(max(common))

    # Evaluate every common run (per-run summary)
    rows = []
    best = None

    def _IC(chi2, k, n):
        AIC = float(chi2 + 2*k)
        BIC = float(chi2 + k*np.log(n))
        return dict(AIC=AIC, BIC=BIC)

    for rid in common:
        pL = dict(runidsL).get(rid, None)
        pE = dict(runidsE).get(rid, None)
        if pL is None or pE is None:
            continue

        thL, lpL, whereL, itL = _map_from_h5(pL, burn=int(N_BURN), thin=int(globals().get("THIN", 1)))
        thE, lpE, whereE, itE = _map_from_h5(pE, burn=int(N_BURN), thin=int(globals().get("THIN", 1)))

        partsL, chi2L = chi2_breakdown_LCDM(thL)
        partsE, chi2E = chi2_breakdown_ECTI(thE)

        chi2L = float(chi2L)
        chi2E = float(chi2E)
        dchi2 = float(chi2E - chi2L)

        IC_L = _IC(chi2L, K_PARAMS, N_DATA)
        IC_E = _IC(chi2E, K_PARAMS, N_DATA)

        rows.append(dict(
            run_id=int(rid),
            chi2_LCDM=chi2L,
            chi2_ECTI=chi2E,
            dchi2=dchi2,
            dAIC=float(IC_E["AIC"] - IC_L["AIC"]),
            dBIC=float(IC_E["BIC"] - IC_L["BIC"]),
        ))

        if rid == rid_ref:
            best = dict(
                policy="LAST_RUN",
                rid_ref=rid_ref,
                LCDM=dict(h5=pL, theta_map=thL.tolist(), logp_map=float(lpL), where=whereL,
                          parts=dict(partsL), chi2=chi2L, IC=IC_L),
                ECTI=dict(h5=pE, theta_map=thE.tolist(), logp_map=float(lpE), where=whereE,
                          parts=dict(partsE), chi2=chi2E, IC=IC_E),
                meta=dict(
                    CAMPAIGN_DIR=CAMPAIGN_DIR,
                    families=dict(LCDM=famL, ECTI=famE),
                    N_DATA=int(N_DATA),
                    k_params=int(K_PARAMS),
                    toggles=dict(
                        USE_KIDS=bool(globals().get("USE_KIDS", False)),
                        USE_PLANCK_LENSING=bool(globals().get("USE_PLANCK_LENSING", False)),
                    ),
                )
            )

    if best is None:
        raise RuntimeError("Failed to build BEST_13 for reference run.")

    df = pd.DataFrame(rows).sort_values("run_id").reset_index(drop=True)

    globals()["FINAL_DF_10.1"] = df
    globals()["BEST_10.1"] = best
    globals()["N_DATA_10.1"] = int(N_DATA)
    globals()["K_PARAMS_10.1"] = int(K_PARAMS)

    # Save artifacts
    csv_path = os.path.join(TAB_DIR, "final_results_per_run_k5.csv")
    json_path = os.path.join(TAB_DIR, "final_results_last_run_k5.json")
    df.to_csv(csv_path, index=False)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2, ensure_ascii=False)

    print("====================================================")
    print("10.1) FINAL RESULTS — FAIR k=5 (LAST RUN POLICY)")
    print("----------------------------------------------------")
    print("CAMPAIGN_DIR =", CAMPAIGN_DIR)
    print("Families     =", f"LCDM:{famL} | ECTI:{famE}")
    print(f"N_data       = {N_DATA} (SN={N_SN}, BAO={N_BAO}, RSD={N_RSD}, KiDS={N_KIDS}, CMB={N_CMB}, LENS={N_LENS})")
    print(f"k_params     = {K_PARAMS}")
    print("----------------------------------------------------")
    print("Per-run summary:")
    print(df.to_string(index=False))
    print("----------------------------------------------------")
    print(f"REFERENCE = LAST common run_id = run{rid_ref:03d}")
    print("LCDM: file=", os.path.basename(best["LCDM"]["h5"]), "| chi2=", f"{best['LCDM']['chi2']:.3f}",
          "| AIC=", f"{best['LCDM']['IC']['AIC']:.3f}", "| BIC=", f"{best['LCDM']['IC']['BIC']:.3f}")
    print("ECTI: file=", os.path.basename(best["ECTI"]["h5"]), "| chi2=", f"{best['ECTI']['chi2']:.3f}",
          "| AIC=", f"{best['ECTI']['IC']['AIC']:.3f}", "| BIC=", f"{best['ECTI']['IC']['BIC']:.3f}")
    print("Δχ²_ref (ECTI−LCDM) =", f"{(best['ECTI']['chi2']-best['LCDM']['chi2']):+.6f}")
    print("====================================================")
    print("✔ Saved:", csv_path)
    print("✔ Saved:", json_path)
    print("✔ Stored: FINAL_DF_10.1, BEST_10.1, N_DATA_10.1, K_PARAMS_10.1")


In [ ]:
# ============================================================
# 10.2) Run-to-run stability (FAIR k=5) — uses FINAL_DF_10.1
#   - prints Δχ² / ΔAIC / ΔBIC stability across paired runs
#   - saves CSV summary
# ============================================================

import os
import numpy as np
import pandas as pd

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_cd = globals().get("CAMPAIGN_DIR", None)
_SKIP_POST = (not _RUN_MCMC) or (not isinstance(_cd, str)) or (not os.path.isdir(_cd))

if _SKIP_POST:
    print("⏭ Skipping Cell 10.2 (stability): RUN_MCMC=False or CAMPAIGN_DIR missing/invalid.")
else:
    if "FINAL_DF_10.1" not in globals():
        print("⏭ Skipping Cell 10.2 (stability): FINAL_DF_13 not found (run Cell 10.1 first).")
        _SKIP_POST = True

if not _SKIP_POST:
    # Rehydrate TAB_DIR (Run-All safe)
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(_cd, "tables"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    df = globals()["FINAL_DF_10.1"].copy()
    if not isinstance(df, pd.DataFrame) or df.empty:
        raise RuntimeError("FINAL_DF_10.1 is missing or empty.")

    # Ensure expected columns exist
    for col in ["run_id", "chi2_LCDM", "chi2_ECTI", "dchi2", "dAIC", "dBIC"]:
        if col not in df.columns:
            raise RuntimeError(f"FINAL_DF_10.1 missing required column: {col}")

    # Summary stats
    dchi = df["dchi2"].astype(float).values
    dAIC = df["dAIC"].astype(float).values
    dBIC = df["dBIC"].astype(float).values

    stats = dict(
        dchi2_mean=float(np.mean(dchi)),
        dchi2_std=float(np.std(dchi, ddof=1)) if dchi.size > 1 else float("nan"),
        dAIC_mean=float(np.mean(dAIC)),
        dAIC_std=float(np.std(dAIC, ddof=1)) if dAIC.size > 1 else float("nan"),
        dBIC_mean=float(np.mean(dBIC)),
        dBIC_std=float(np.std(dBIC, ddof=1)) if dBIC.size > 1 else float("nan"),
        n_runs=int(dchi.size),
    )

    out = df[["run_id", "chi2_LCDM", "chi2_ECTI", "dchi2", "dAIC", "dBIC"]].copy()
    csv_path = os.path.join(TAB_DIR_LOCAL, "run_to_run_stability_k5.csv")
    out.to_csv(csv_path, index=False)

    print("====================================================")
    print("10.2) Run-to-run stability (FAIR k=5)")
    print("----------------------------------------------------")
    print(f"CAMPAIGN_DIR = {globals().get('CAMPAIGN_DIR')}")
    print(out.to_string(index=False))
    print("----------------------------------------------------")
    print("Summary stats:")
    for k, v in stats.items():
        print(f"  {k:12s} = {v}")
    print("----------------------------------------------------")
    print("✔ Saved:", csv_path)
    print("====================================================")

    globals()["STABILITY_STATS_10.2"] = stats
    globals()["STABILITY_DF_10.2"] = out
    print("✔ Stored: STABILITY_STATS_10.2, STABILITY_DF_10.2")


In [ ]:
# ============================================================
# 10.3) Combined posterior (p16/p50/p84) — FAIR k=5
#   - Reads ALL H5 in CAMPAIGN_DIR for LCDM_k5 and ECTI_k5
#   - Concatenates post-burn samples (flat)
#   - Exports quantiles + combined samples to NPZ
# ============================================================

import os, re
import numpy as np
import pandas as pd
from emcee.backends import HDFBackend

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_cd = globals().get("CAMPAIGN_DIR", None)
_SKIP_POST = (not _RUN_MCMC) or (not isinstance(_cd, str)) or (not os.path.isdir(_cd))

if _SKIP_POST:
    print("⏭ Skipping Cell 10.3 (combined posterior): RUN_MCMC=False or CAMPAIGN_DIR missing/invalid.")
else:
    if not any(f.lower().endswith(".h5") for f in os.listdir(_cd)):
        _SKIP_POST = True
        print("⏭ Skipping Cell 10.3 (combined posterior): no .h5 files found in CAMPAIGN_DIR=" + str(_cd))

if not _SKIP_POST:

    # Required symbols
    _required = ["CAMPAIGN_DIR", "N_BURN", "THIN"]
    _missing = [k for k in _required if k not in globals()]
    if _missing:
        raise NameError("Missing required symbols before 10.3: " + ", ".join(_missing))

    # Output dirs under campaign (standard, repo-friendly)
    TAB_DIR_LOCAL  = globals().get("TAB_DIR",  os.path.join(_cd, "tables"))
    POST_DIR_LOCAL = globals().get("POST_DIR", os.path.join(_cd, "posteriors"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    os.makedirs(POST_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"] = TAB_DIR_LOCAL
    globals()["POST_DIR"] = POST_DIR_LOCAL

    PARAM_NAMES = ["H0", "Om0", "sigma8", "M", "wb"]

    def _list_family_files(campaign_dir):
        files = [f for f in os.listdir(campaign_dir) if f.lower().endswith(".h5")]
        pat = re.compile(r"^(.*)_run(\d+)\.h5$", re.IGNORECASE)
        buckets = {}
        for f in files:
            m = pat.match(f)
            if not m:
                continue
            fam = m.group(1).upper()
            rid = int(m.group(2))
            if not (fam.startswith("LCDM") or fam.startswith("ECTI")):
                continue
            buckets.setdefault(fam, []).append((rid, os.path.join(campaign_dir, f)))
        for fam in list(buckets.keys()):
            buckets[fam] = [p for _, p in sorted(buckets[fam], key=lambda x: x[0])]
        return buckets

    def _choose_family(buckets, kind="LCDM"):
        keys = sorted([k for k in buckets.keys() if k.startswith(kind.upper())])
        if not keys:
            return None
        pref = f"{kind.upper()}_K5"
        for k in keys:
            if pref in k:
                return k
        return keys[0]

    def _read_flat_samples(h5_path, burn, thin):
        b = HDFBackend(h5_path)
        it = int(getattr(b, "iteration", b.get_chain().shape[0]))
        if it <= 0:
            raise RuntimeError(f"H5 has no samples: {os.path.basename(h5_path)}")

        burn_eff = int(min(int(burn), max(it - 1, 0)))
        if it <= burn_eff:
            raise RuntimeError(
                f"H5 iteration={it} <= burn_eff={burn_eff} (post-burn empty): {os.path.basename(h5_path)}"
            )

        flat = b.get_chain(discard=burn_eff, thin=int(thin), flat=True)
        flat = np.asarray(flat, dtype=float)

        if flat.ndim != 2 or flat.shape[1] != 5:
            raise ValueError(
                f"Expected flat chain (Nsamp,5) in {os.path.basename(h5_path)}, got {flat.shape}"
            )
        if flat.shape[0] < 1:
            raise RuntimeError(f"Empty flat chain post-burn in {os.path.basename(h5_path)}")
        return flat

    def _quantiles(S):
        q16 = np.percentile(S, 16, axis=0)
        q50 = np.percentile(S, 50, axis=0)
        q84 = np.percentile(S, 84, axis=0)
        return q16, q50, q84

    buckets = _list_family_files(CAMPAIGN_DIR)
    famL = _choose_family(buckets, "LCDM")
    famE = _choose_family(buckets, "ECTI")
    if famL is None or famE is None:
        raise RuntimeError(f"Families not found in CAMPAIGN_DIR. Found={sorted(buckets.keys())}")

    filesL = buckets[famL]
    filesE = buckets[famE]
    if not filesL or not filesE:
        raise RuntimeError(f"Empty H5 family lists: LCDM={len(filesL)} ECTI={len(filesE)}")

    samplesL = [ _read_flat_samples(p, burn=int(N_BURN), thin=int(THIN)) for p in filesL ]
    samplesE = [ _read_flat_samples(p, burn=int(N_BURN), thin=int(THIN)) for p in filesE ]

    S_L = np.vstack(samplesL) if len(samplesL) else np.empty((0,5), float)
    S_E = np.vstack(samplesE) if len(samplesE) else np.empty((0,5), float)

    if S_L.shape[0] < 10 or S_E.shape[0] < 10:
        raise RuntimeError(f"Too few post-burn samples: LCDM={S_L.shape[0]}, ECTI={S_E.shape[0]}")

    qL16, qL50, qL84 = _quantiles(S_L)
    qE16, qE50, qE84 = _quantiles(S_E)

    dfq = pd.DataFrame({
        "param": PARAM_NAMES,
        "LCDM_p16": qL16, "LCDM_p50": qL50, "LCDM_p84": qL84,
        "ECTI_p16": qE16, "ECTI_p50": qE50, "ECTI_p84": qE84,
    })

    print("====================================================")
    print("10.3) Combined posterior quantiles (FAIR k=5)")
    print("----------------------------------------------------")
    print("CAMPAIGN_DIR =", CAMPAIGN_DIR)
    print(f"LCDM family={famL} | files={len(filesL)} | samples={S_L.shape[0]}")
    print(f"ECTI family={famE} | files={len(filesE)} | samples={S_E.shape[0]}")
    print("----------------------------------------------------")
    print(dfq.to_string(index=False))
    print("====================================================")

    csv_path = os.path.join(TAB_DIR_LOCAL, "posterior_quantiles_combined_k5.csv")
    dfq.to_csv(csv_path, index=False)

    npz_path = os.path.join(POST_DIR_LOCAL, "combined_posteriors_k5.npz")
    np.savez_compressed(
        npz_path,
        param_names=np.array(PARAM_NAMES, dtype=object),
        samples_LCDM=S_L,
        samples_ECTI=S_E,
        q_LCDM=np.vstack([qL16, qL50, qL84]),
        q_ECTI=np.vstack([qE16, qE50, qE84]),
        family_LCDM=famL,
        family_ECTI=famE,
        CAMPAIGN_DIR=CAMPAIGN_DIR,
        N_BURN=int(N_BURN),
        THIN=int(THIN),
    )

    globals()["POST_SAMPLES_LCDM_10.3"] = S_L
    globals()["POST_SAMPLES_ECTI_10.3"] = S_E
    globals()["POST_QUANTILES_10.3"] = dfq

    print("✔ Saved:", csv_path)
    print("✔ Saved:", npz_path)
    print("✔ Stored: POST_SAMPLES_*_10.3, POST_QUANTILES_10.3")


In [ ]:
# ============================================================
# 10.4) Ablations / robustness (subset χ²) @ BEST_10.1 reference MAP — FAIR k=5
#   - NO refit, NO rerun MCMC: evaluates subset totals at the same reference MAP
#   - Saves CSV in TAB_DIR
# ============================================================

import os
import numpy as np
import pandas as pd

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_cd = globals().get("CAMPAIGN_DIR", None)
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 10.4 (ablations): RUN_MCMC=False.")
else:
    if "BEST_10.1" not in globals():
        print("⏭ Skipping Cell 10.4 (ablations): BEST_10.1 not found (run Cell 10.1 first).")
        _SKIP_POST = True

if not _SKIP_POST:

    best = globals()["BEST_10.1"]
    partsL = dict(best["LCDM"]["parts"])
    partsE = dict(best["ECTI"]["parts"])

    # Output dir
    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "tables"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    # Include LENS if present
    ALL_KEYS = ["SN", "BAO", "RSD", "KiDS", "CMB", "LENS"]
    present = [k for k in ALL_KEYS if (k in partsL) or (k in partsE)]

    def _sum_parts(parts, keys):
        return float(sum(float(parts.get(k, 0.0)) for k in keys))

    ablations = [
        ("ALL",                 present),
        ("NO_CMB",              [k for k in present if k != "CMB"]),
        ("NO_LENS",             [k for k in present if k != "LENS"]),
        ("NO_RSD",              [k for k in present if k != "RSD"]),
        ("NO_KIDS",             [k for k in present if k != "KiDS"]),
        ("BG_ONLY_SN+BAO+CMB",  [k for k in ["SN", "BAO", "CMB"] if k in present]),
    ]

    rows = []
    for name, keys in ablations:
        chi2L = _sum_parts(partsL, keys)
        chi2E = _sum_parts(partsE, keys)
        rows.append({
            "ablation": name,
            "keys": "+".join(keys),
            "chi2_LCDM_subset": chi2L,
            "chi2_ECTI_subset": chi2E,
            "dchi2_subset (E-L)": float(chi2E - chi2L),
        })

    df = pd.DataFrame(rows)

    chi2L_all = float(df.loc[df["ablation"]=="ALL","chi2_LCDM_subset"].iloc[0])
    chi2E_all = float(df.loc[df["ablation"]=="ALL","chi2_ECTI_subset"].iloc[0])
    df["dchi2_LCDM_vs_ALL"] = df["chi2_LCDM_subset"] - chi2L_all
    df["dchi2_ECTI_vs_ALL"] = df["chi2_ECTI_subset"] - chi2E_all

    print("====================================================")
    print("10.4) Ablations @ BEST_10.1 reference MAP (NO refit)")
    print("----------------------------------------------------")
    print("Probes present:", present)
    print(df.to_string(index=False))
    print("----------------------------------------------------")
    print("NOTE: subset totals evaluated at SAME reference MAP (no re-optimization).")
    print("====================================================")

    csv_path = os.path.join(TAB_DIR_LOCAL, "ablations_subset_chi2_at_reference_MAP_k5.csv")
    df.to_csv(csv_path, index=False)
    globals()["ABLATIONS_DF_10.4"] = df

    print("✔ Saved:", csv_path)
    print("✔ Stored: ABLATIONS_DF_10.4")


In [ ]:
# ============================================================
# 10.5) Summary figure — Δχ² per probe (ECTI - LCDM) @ BEST_10.1 reference MAP
#   - One "wow" figure that summarizes where the model wins/loses
#   - Saves PNG + CSV
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_cd = globals().get("CAMPAIGN_DIR", None)
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 10.5 (Δχ² per probe fig): RUN_MCMC=False.")
else:
    if "BEST_10.1" not in globals():
        print("⏭ Skipping Cell 10.5: BEST_10.1 not found (run Cell 10.1 first).")
        _SKIP_POST = True

if not _SKIP_POST:

    best = globals()["BEST_10.1"]
    partsL = dict(best["LCDM"]["parts"])
    partsE = dict(best["ECTI"]["parts"])

    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    FIG_DIR_LOCAL = globals().get("FIG_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "figures"))
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "tables"))
    os.makedirs(FIG_DIR_LOCAL, exist_ok=True)
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["FIG_DIR"] = FIG_DIR_LOCAL
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    # include LENS if present
    order = ["SN", "BAO", "RSD", "KiDS", "CMB", "LENS"]
    keys = [k for k in order if (k in partsL) or (k in partsE)]

    df = pd.DataFrame({
        "probe": keys,
        "chi2_LCDM": [float(partsL.get(k, 0.0)) for k in keys],
        "chi2_ECTI": [float(partsE.get(k, 0.0)) for k in keys],
    })
    df["dchi2 (ECTI-LCDM)"] = df["chi2_ECTI"] - df["chi2_LCDM"]
    df["abs"] = np.abs(df["dchi2 (ECTI-LCDM)"])
    df_sorted = df.sort_values("abs", ascending=False).drop(columns=["abs"]).reset_index(drop=True)

    csv_path = os.path.join(TAB_DIR_LOCAL, "delta_chi2_per_probe_k5.csv")
    df_sorted.to_csv(csv_path, index=False)

    fig_path = os.path.join(FIG_DIR_LOCAL, "delta_chi2_per_probe_k5.png")
    plt.figure(figsize=(7.8, 4.6))
    x = np.arange(len(df_sorted))
    vals = df_sorted["dchi2 (ECTI-LCDM)"].values
    plt.bar(x, vals)
    plt.axhline(0.0)
    plt.xticks(x, df_sorted["probe"].values)
    plt.ylabel(r"$\Delta \chi^2$ (ECTI $-$ $\Lambda$CDM)")
    plt.title(r"Where ECTI wins/loses (reference MAP, FAIR $k=5$)")

    for i, v in enumerate(vals):
        plt.text(i, v, f"{v:+.2f}", ha="center", va=("bottom" if v >= 0 else "top"))

    plt.tight_layout()
    plt.savefig(fig_path, dpi=180)
    plt.close()

    globals()["DCHI2_PER_PROBE_DF_10.5"] = df_sorted

    print("✔ Saved:", csv_path)
    print("✔ Saved:", fig_path)
    print("✔ Stored: DCHI2_PER_PROBE_DF_10.5")
    print(df_sorted.to_string(index=False))


# 11) Publication-ready tables (LaTeX)

This section generates **LaTeX tables** summarizing the final results
for direct inclusion in an article, report, or arXiv submission.

Two tables are produced:

1. Best-fit χ² breakdown by dataset (ΛCDM vs ECTI-P)
2. Model comparison using χ², AIC, and BIC

The output can be copied directly into a LaTeX document
without any modification.

In [ ]:
# ============================================================
# 11.1) LaTeX tables for publication — FAIR k=5 (BEST_10.1 reference MAP)
#   - χ² breakdown table (includes LENS if present)
#   - IC (AIC/BIC) table (from BEST_10.1)
#   - Saves .tex into TAB_DIR
# ============================================================

import os
import numpy as np

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_cd = globals().get("CAMPAIGN_DIR", None)
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 11.1 (LaTeX tables): RUN_MCMC=False.")
else:
    if "BEST_10.1" not in globals():
        print("⏭ Skipping Cell 11.1: BEST_10.1 not found (run Cell 10.1 first).")
        _SKIP_POST = True

if not _SKIP_POST:

    best = globals()["BEST_10.1"]

    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "tables"))
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    partsL = dict(best["LCDM"]["parts"])
    partsE = dict(best["ECTI"]["parts"])

    IC_L = dict(best["LCDM"].get("IC", {}))
    IC_E = dict(best["ECTI"].get("IC", {}))

    def latex_float(x, ndigits=3):
        return f"{float(x):.{ndigits}f}"

    # -----------------------------
    # Table 1: χ² breakdown
    # -----------------------------
    order = ["SN", "BAO", "RSD", "KiDS", "CMB", "LENS"]
    keys = [k for k in order if (k in partsL) or (k in partsE)]
    chi2L_tot = float(best["LCDM"]["chi2"])
    chi2E_tot = float(best["ECTI"]["chi2"])

    rows = []
    for k in keys:
        rows.append((k, partsL.get(k, 0.0), partsE.get(k, 0.0)))

    latex_chi2 = []
    latex_chi2.append(r"\begin{table}[h]")
    latex_chi2.append(r"\centering")
    latex_chi2.append(r"\caption{Best-fit $\chi^2$ contributions by dataset (reference MAP, FAIR $k=5$).}")
    latex_chi2.append(r"\begin{tabular}{lcc}")
    latex_chi2.append(r"\hline")
    latex_chi2.append(r"Dataset & $\Lambda$CDM & ECTI-P \\")
    latex_chi2.append(r"\hline")
    for k, vL, vE in rows:
        latex_chi2.append(f"{k} & {latex_float(vL)} & {latex_float(vE)} \\\\")
    latex_chi2.append(r"\hline")
    latex_chi2.append(f"Total & {latex_float(chi2L_tot)} & {latex_float(chi2E_tot)} \\\\")
    latex_chi2.append(r"\hline")
    latex_chi2.append(r"\end{tabular}")
    latex_chi2.append(r"\end{table}")
    latex_chi2_str = "\n".join(latex_chi2)

    # -----------------------------
    # Table 2: Information criteria
    # -----------------------------
    # Note: AIC/BIC computed in Cell 10.1; here we only format/report.
    aicL = IC_L.get("AIC", np.nan)
    bicL = IC_L.get("BIC", np.nan)
    aicE = IC_E.get("AIC", np.nan)
    bicE = IC_E.get("BIC", np.nan)

    latex_ic = []
    latex_ic.append(r"\begin{table}[h]")
    latex_ic.append(r"\centering")
    latex_ic.append(r"\caption{Information criteria at the reference MAP (FAIR $k=5$).}")
    latex_ic.append(r"\begin{tabular}{lcc}")
    latex_ic.append(r"\hline")
    latex_ic.append(r"Metric & $\Lambda$CDM & ECTI-P \\")
    latex_ic.append(r"\hline")
    latex_ic.append(f"AIC & {latex_float(aicL)} & {latex_float(aicE)} \\\\")
    latex_ic.append(f"BIC & {latex_float(bicL)} & {latex_float(bicE)} \\\\")
    latex_ic.append(r"\hline")
    latex_ic.append(r"\end{tabular}")
    latex_ic.append(r"\end{table}")
    latex_ic_str = "\n".join(latex_ic)

    out1 = os.path.join(TAB_DIR_LOCAL, "table_chi2_breakdown_k5.tex")
    out2 = os.path.join(TAB_DIR_LOCAL, "table_information_criteria_k5.tex")
    with open(out1, "w", encoding="utf-8") as f:
        f.write(latex_chi2_str + "\n")
    with open(out2, "w", encoding="utf-8") as f:
        f.write(latex_ic_str + "\n")

    print("✔ Saved:", out1)
    print("✔ Saved:", out2)


# 12) Posterior distributions (corner plots)

This section produces **corner plots** of the posterior distributions
for both ΛCDM and ECTI-P using the converged MCMC chains.

These plots are intended for:
- visual inspection of parameter degeneracies,
- supplementary material (paper / arXiv),
- sanity checks against pathological posteriors.

The plots are generated **only if MCMC chains are already available**.
No MCMC run is triggered in this section.

In [ ]:
# ============================================================
# 12.1) Corner plot — FAIR k=5 (combined posterior if available)
#   - Prefers combined samples from Cell 10.3 (POST_SAMPLES_*_10.3)
#   - Falls back to LAST run H5 (BEST_10.1) if combined samples missing
#   - Saves PNG into FIG_DIR
# ============================================================

import os
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 12.1 (corner): RUN_MCMC=False.")
else:
    # Need either combined samples OR BEST_10.1 to fallback
    if ("POST_SAMPLES_LCDM_10.3" not in globals()) and ("BEST_10.1" not in globals()):
        print("⏭ Skipping Cell 12.1 (corner): need POST_SAMPLES_*_10.3 or BEST_10.1 (run Cells 10.1/10.3 first).")
        _SKIP_POST = True

if not _SKIP_POST:

    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    FIG_DIR_LOCAL = globals().get("FIG_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "figures"))
    os.makedirs(FIG_DIR_LOCAL, exist_ok=True)
    globals()["FIG_DIR"] = FIG_DIR_LOCAL

    # Optional dependency: corner
    try:
        import corner
        import matplotlib.lines as mlines
        _has_corner = True
    except Exception:
        _has_corner = False

    PARAM_NAMES = ["H0", "Om0", "sigma8", "M", "wb"]

    def _load_from_best_10_1(model_key: str) -> np.ndarray:
        """
        Fallback loader: reads LAST-run H5 paths stored in globals()['BEST_10.1'].
        Returns a (Nsamp, 5) flat chain after discard/thin.
        """
        from emcee.backends import HDFBackend

        if "BEST_10.1" not in globals():
            raise NameError("BEST_10.1 not found in globals() (run Cell 10.1 first).")

        best = globals()["BEST_10.1"]
        if model_key not in best or "h5" not in best[model_key]:
            raise KeyError(f"BEST_10.1 missing key '{model_key}[\"h5\"]'.")

        h5 = best[model_key]["h5"]
        b = HDFBackend(h5)

        it = int(getattr(b, "iteration", b.get_chain().shape[0]))
        burn = int(globals().get("N_BURN", 0))
        thin = int(globals().get("THIN", 1))
        burn_eff = int(min(burn, max(it - 1, 0)))

        if it <= burn_eff:
            raise RuntimeError(f"H5 iteration={it} <= burn_eff={burn_eff}: {os.path.basename(h5)}")

        flat = b.get_chain(discard=burn_eff, thin=thin, flat=True)
        flat = np.asarray(flat, float)

        if flat.ndim != 2 or flat.shape[1] != 5:
            raise RuntimeError(f"Expected (Nsamp,5) chain in {os.path.basename(h5)}, got {flat.shape}")

        return flat

    # -----------------------------
    # Load samples
    # -----------------------------
    if ("POST_SAMPLES_LCDM_10.3" in globals()) and ("POST_SAMPLES_ECTI_10.3" in globals()):
        S_L = np.asarray(globals()["POST_SAMPLES_LCDM_10.3"], float)
        S_E = np.asarray(globals()["POST_SAMPLES_ECTI_10.3"], float)
        source = "combined posterior (10.3)"
    else:
        S_L = _load_from_best_10_1("LCDM")
        S_E = _load_from_best_10_1("ECTI")
        source = "LAST run H5 (BEST_10.1)"

    if S_L.ndim != 2 or S_L.shape[1] != 5:
        raise RuntimeError(f"Expected LCDM samples shape (Nsamp,5), got {S_L.shape}")
    if S_E.ndim != 2 or S_E.shape[1] != 5:
        raise RuntimeError(f"Expected ECTI samples shape (Nsamp,5), got {S_E.shape}")

    if S_L.shape[0] < 100 or S_E.shape[0] < 100:
        raise RuntimeError(f"Too few samples for corner: LCDM={S_L.shape[0]}, ECTI={S_E.shape[0]}")

    out_png = os.path.join(FIG_DIR_LOCAL, "corner_LCDM_vs_ECTI_k5.png")

    # -----------------------------
    # Plot
    # -----------------------------
    if not _has_corner:
        print("⚠ 'corner' not installed. Skipping corner plot generation.")
    else:
        # Requested visual distinction: ΛCDM = blue, ECTI = red
        fig = corner.corner(
            S_L,
            labels=PARAM_NAMES,
            show_titles=True,
            title_fmt=".3f",
            color="C0",
            plot_datapoints=False,
            fill_contours=False,
        )
        corner.corner(
            S_E,
            labels=PARAM_NAMES,
            show_titles=True,
            title_fmt=".3f",
            fig=fig,
            color="C3",
            plot_datapoints=False,
            fill_contours=False,
        )

        # Legend (top-right)
        h_lcdm = mlines.Line2D([], [], color="C0", label="ΛCDM")
        h_ecti = mlines.Line2D([], [], color="C3", label="ECTI-P")
        fig.legend(handles=[h_lcdm, h_ecti], loc="upper right", frameon=False)

        fig.savefig(out_png, dpi=180)
        plt.close(fig)
        print("✔ Saved:", out_png)
        print("Source:", source)


# 13) Residual plots: SN Ia, BAO, and RSD

In this section, we compare the performance of ΛCDM and ECTI-P
directly at the level of observables by inspecting residuals.

Residuals are defined as:
- Supernovae:     Δμ(z) = μ_obs − μ_th
- BAO:            Δ(D_V / r_s)
- RSD:            Δ(fσ₈)

These plots provide an intuitive, model-independent view of
where each model succeeds or fails relative to the data.

All residuals are computed using the best-fit parameters
obtained from the converged MCMC runs (R̂-validated).

In [ ]:
# ============================================================
# 13.1) Residuals / sanity plots @ BEST_10.1 reference MAP — FAIR k=5 (LAST run)
#   - SN residuals: (mu_obs - mu_model)/sigma
#   - BAO pulls: (obs - theory)/sigma (diag from cov)
#   - RSD pulls: (obs - pred)/err
#   - Saves figures + CSV
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 13.1 (residuals): RUN_MCMC=False.")
else:
    # need BEST_10.1 + core data arrays + model eval fns
    required = [
        "BEST_10.1",
        "z_SN", "mu_SN", "Cinv_SN",
        "BAO_obs", "BAO_inv_cov",
        "z_RSD", "fs8_obs", "fs8_err",
        "mu_LCDM", "mu_ECTI",
        "bao_theory_vector_LCDM", "bao_theory_vector_ECTI",
        "fsigma8_theory",
    ]
    missing = [k for k in required if k not in globals()]
    if missing:
        print("⏭ Skipping Cell 13.1 (residuals): missing required symbols:", missing)
        _SKIP_POST = True

if not _SKIP_POST:

    best = globals()["BEST_10.1"]
    thetaL = np.array(best["LCDM"]["theta_map"], dtype=float).ravel()
    thetaE = np.array(best["ECTI"]["theta_map"], dtype=float).ravel()

    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    FIG_DIR_LOCAL = globals().get("FIG_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "figures"))
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "tables"))
    os.makedirs(FIG_DIR_LOCAL, exist_ok=True)
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    globals()["FIG_DIR"] = FIG_DIR_LOCAL
    globals()["TAB_DIR"] = TAB_DIR_LOCAL

    # ------------------------------------------------------------
    # SN residuals
    # ------------------------------------------------------------
    zsn = np.asarray(globals()["z_SN"], float).ravel()
    mu_obs = np.asarray(globals()["mu_SN"], float).ravel()

    muL = np.asarray(mu_LCDM(zsn, thetaL), float).ravel()
    muE = np.asarray(mu_ECTI(zsn, thetaE), float).ravel()

    Cinv = np.asarray(globals()["Cinv_SN"], float)
    C = np.linalg.inv(Cinv)
    sig_sn = np.sqrt(np.diag(C))

    resL = (mu_obs - muL) / sig_sn
    resE = (mu_obs - muE) / sig_sn

    df_sn = pd.DataFrame({"z": zsn, "res_LCDM": resL, "res_ECTI": resE, "sigma_mu": sig_sn})
    df_sn.to_csv(os.path.join(TAB_DIR_LOCAL, "residuals_SN.csv"), index=False)

    plt.figure(figsize=(7, 3.0))
    plt.axhline(0.0, lw=0.8)
    plt.scatter(zsn, resL, s=8, label=r"$\Lambda$CDM")
    plt.scatter(zsn, resE, s=8, label="ECTI-P")
    plt.xlabel(r"$z$")
    plt.ylabel(r"$(\mu_{\rm obs}-\mu_{\rm model})/\sigma$")
    plt.title("SN residuals (Pantheon+)")
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR_LOCAL, "residuals_SN.png"), dpi=180)
    plt.close()

    # ------------------------------------------------------------
    # BAO pulls
    # ------------------------------------------------------------
    ybao = np.asarray(globals()["BAO_obs"], float).ravel()

    tL = np.asarray(bao_theory_vector_LCDM(thetaL), float).ravel()
    tE = np.asarray(bao_theory_vector_ECTI(thetaE), float).ravel()

    if tL.size != ybao.size or tE.size != ybao.size:
        raise RuntimeError(f"BAO theory size mismatch: obs={ybao.size} LCDM={tL.size} ECTI={tE.size}")

    invcov_bao = np.asarray(globals()["BAO_inv_cov"], float)
    cov_bao = np.linalg.inv(invcov_bao)
    sig_bao = np.sqrt(np.diag(cov_bao))

    pullL_bao = (ybao - tL) / sig_bao
    pullE_bao = (ybao - tE) / sig_bao

    df_bao = pd.DataFrame({
        "i": np.arange(len(ybao)),
        "pull_LCDM": pullL_bao,
        "pull_ECTI": pullE_bao,
        "sigma": sig_bao,
        "obs": ybao,
        "th_LCDM": tL,
        "th_ECTI": tE,
    })
    df_bao.to_csv(os.path.join(TAB_DIR_LOCAL, "pulls_BAO.csv"), index=False)

    plt.figure(figsize=(7, 3.0))
    plt.axhline(0.0, lw=0.8)
    x = np.arange(len(ybao))
    plt.plot(x, pullL_bao, marker="o", label=r"$\Lambda$CDM")
    plt.plot(x, pullE_bao, marker="o", label="ECTI-P")
    plt.xlabel("BAO point index")
    plt.ylabel(r"$(\mathrm{obs}-\mathrm{th})/\sigma$")
    plt.title("BAO pulls (DESI 2024)")
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR_LOCAL, "pulls_BAO.png"), dpi=180)
    plt.close()

    # ------------------------------------------------------------
    # RSD pulls (fσ8)
    # ------------------------------------------------------------
    zrsd = np.asarray(globals()["z_RSD"], float).ravel()
    obs_rsd = np.asarray(globals()["fs8_obs"], float).ravel()
    err_rsd = np.asarray(globals()["fs8_err"], float).ravel()

    # fsigma8_theory signature observed in your notebook: fsigma8_theory(theta, model='LCDM')
    predL = np.asarray(fsigma8_theory(thetaL, model="LCDM"), float).ravel()
    predE = np.asarray(fsigma8_theory(thetaE, model="ECTI"), float).ravel()

    if predL.size != obs_rsd.size or predE.size != obs_rsd.size:
        raise RuntimeError(f"RSD size mismatch: obs={obs_rsd.size} LCDM={predL.size} ECTI={predE.size}")

    pullL_rsd = (obs_rsd - predL) / err_rsd
    pullE_rsd = (obs_rsd - predE) / err_rsd

    df_rsd = pd.DataFrame({
        "z": zrsd,
        "pull_LCDM": pullL_rsd,
        "pull_ECTI": pullE_rsd,
        "obs": obs_rsd,
        "err": err_rsd,
        "th_LCDM": predL,
        "th_ECTI": predE,
    })
    df_rsd.to_csv(os.path.join(TAB_DIR_LOCAL, "pulls_RSD.csv"), index=False)

    plt.figure(figsize=(7, 3.0))
    plt.axhline(0.0, lw=0.8)
    plt.plot(zrsd, pullL_rsd, marker="o", label=r"$\Lambda$CDM")
    plt.plot(zrsd, pullE_rsd, marker="o", label="ECTI-P")
    plt.xlabel(r"$z$")
    plt.ylabel(r"$(f\sigma_8^{\rm obs}-f\sigma_8^{\rm th})/\sigma$")
    plt.title("RSD pulls")
    plt.legend(frameon=False)
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR_LOCAL, "pulls_RSD.png"), dpi=180)
    plt.close()

    print("=== Cell 13.1 done ===")
    print("Saved figures to:", FIG_DIR_LOCAL)
    print("Saved CSV to:", TAB_DIR_LOCAL)


In [ ]:
# ============================================================
# 13.2) Referee checklist — reproducibility & coherence (FAIR k=5, LAST run)
#   - Hard checks (skip-safe when RUN_MCMC=False)
# ============================================================

import os
import numpy as np

# -----------------------------
# Run-All guard (skip-safe)
# -----------------------------
_RUN_MCMC = bool(globals().get("RUN_MCMC", True))
_SKIP_POST = (not _RUN_MCMC)

if _SKIP_POST:
    print("⏭ Skipping Cell 13.2 (referee checklist): RUN_MCMC=False.")
else:
    _required = [
        "BEST_10.1",
        "CMB_OBS", "CMB_INV_COV",
        "chi2_breakdown_LCDM", "chi2_breakdown_ECTI",
    ]
    _missing = [k for k in _required if k not in globals()]
    if _missing:
        print("⏭ Skipping Cell 13.2 (referee checklist): missing required symbols:", _missing)
        _SKIP_POST = True

if not _SKIP_POST:

    best = globals()["BEST_10.1"]
    thL = np.asarray(best["LCDM"]["theta_map"], float).ravel()
    thE = np.asarray(best["ECTI"]["theta_map"], float).ravel()

    def _assert(cond, msg):
        if not bool(cond):
            raise AssertionError(msg)

    # 0) Policy label
    policy = str(best.get("policy", "UNKNOWN"))
    rid_ref = best.get("rid_ref", None)

    # 1) FAIR theta sizes
    _assert(thL.size == 5, f"LCDM theta at reference MAP must be θ5, got size {thL.size}")
    _assert(thE.size == 5, f"ECTI theta at reference MAP must be θ5, got size {thE.size}")

    # 2) Planck lite 3D shapes
    OBS = np.asarray(globals()["CMB_OBS"], float).ravel()
    INV = np.asarray(globals()["CMB_INV_COV"], float)
    _assert(OBS.size == 3, f"CMB_OBS must have length 3 (R,lA,wb), got {OBS.size}")
    _assert(INV.shape == (3,3), f"CMB_INV_COV must be 3x3, got {INV.shape}")

    if "CMB_COV" in globals():
        COV = np.asarray(globals()["CMB_COV"], float)
        _assert(COV.shape == (3,3), f"CMB_COV must be 3x3, got {COV.shape}")
    else:
        try:
            COV = np.linalg.inv(INV)
        except Exception:
            COV = np.linalg.pinv(INV)
        _assert(COV.shape == (3,3), f"Reconstructed CMB_COV must be 3x3, got {COV.shape}")

    def _max_abs(A):
        A = np.asarray(A, float)
        return float(np.max(np.abs(A))) if A.size else 0.0

    sym_INV = _max_abs(INV - INV.T)
    sym_COV = _max_abs(COV - COV.T)
    _assert(sym_INV <= 1e-12 * (1.0 + _max_abs(INV)), f"CMB_INV_COV not symmetric enough: {sym_INV:.3e}")
    _assert(sym_COV <= 1e-12 * (1.0 + _max_abs(COV)), f"CMB_COV not symmetric enough: {sym_COV:.3e}")

    I = np.eye(3)
    prod = COV @ INV
    errI = _max_abs(prod - I)
    _assert(errI <= 1e-8 * (1.0 + _max_abs(I)), f"COV @ INV not close to I: {errI:.3e}")

    # 3) chi2 coherence
    partsL, sumL = globals()["chi2_breakdown_LCDM"](thL)
    partsE, sumE = globals()["chi2_breakdown_ECTI"](thE)
    partsL = dict(partsL)
    partsE = dict(partsE)

    if "chi2_tot_LCDM" in globals() and callable(globals()["chi2_tot_LCDM"]):
        totL = float(globals()["chi2_tot_LCDM"](thL))
        _assert(abs(totL - float(sumL)) <= 1e-6*(1.0+abs(float(sumL))),
                f"LCDM: chi2_tot != sum(parts): {totL} vs {float(sumL)}")
    else:
        totL = float(sumL)

    if "chi2_tot_ECTI" in globals() and callable(globals()["chi2_tot_ECTI"]):
        totE = float(globals()["chi2_tot_ECTI"](thE))
        _assert(abs(totE - float(sumE)) <= 1e-6*(1.0+abs(float(sumE))),
                f"ECTI: chi2_tot != sum(parts): {totE} vs {float(sumE)}")
    else:
        totE = float(sumE)

    _assert(abs(float(sum(partsL.values())) - float(sumL)) <= 1e-9*(1.0+abs(float(sumL))),
            "LCDM: sum(parts dict) != reported total from chi2_breakdown_LCDM")
    _assert(abs(float(sum(partsE.values())) - float(sumE)) <= 1e-9*(1.0+abs(float(sumE))),
            "ECTI: sum(parts dict) != reported total from chi2_breakdown_ECTI")

    # 4) Output dirs exist
    CAMPAIGN_DIR_LOCAL = globals().get("CAMPAIGN_DIR", os.getcwd())
    FIG_DIR_LOCAL = globals().get("FIG_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "figures"))
    TAB_DIR_LOCAL = globals().get("TAB_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "tables"))
    POST_DIR_LOCAL = globals().get("POST_DIR", os.path.join(CAMPAIGN_DIR_LOCAL, "posteriors"))
    os.makedirs(FIG_DIR_LOCAL, exist_ok=True)
    os.makedirs(TAB_DIR_LOCAL, exist_ok=True)
    os.makedirs(POST_DIR_LOCAL, exist_ok=True)

    _assert(os.path.isdir(FIG_DIR_LOCAL), "FIG_DIR missing (could not create).")
    _assert(os.path.isdir(TAB_DIR_LOCAL), "TAB_DIR missing (could not create).")
    _assert(os.path.isdir(POST_DIR_LOCAL), "POST_DIR missing (could not create).")

    # 5) Early physics hooks present
    has_rs = ("get_rs_star" in globals() and callable(globals()["get_rs_star"])) or ("rs_star" in globals() and callable(globals()["rs_star"]))
    _assert(has_rs, "Missing sound-horizon function: expected callable get_rs_star or rs_star.")

    # Print summary
    print("====================================================")
    print("13.2) Referee checklist — PASSED")
    print("----------------------------------------------------")
    tag = f"Policy: {policy}"
    if (policy.upper() == "LAST_RUN") and (rid_ref is not None):
        tag += f" | run{int(rid_ref):03d}"
    print(tag)
    print("FAIR θ5 sizes: OK")
    print("Planck lite 3D shapes: OK (R,lA,wb + 3x3 cov/inv_cov)")
    print("CMB cov/inv_cov symmetry + COV@INV≈I: OK")
    print("chi2 coherence: chi2_tot == sum(parts): OK")
    print("Output directories exist: OK")
    print("Early-time physics hooks present: OK")
    print("----------------------------------------------------")
    print("Reference MAP summary (BEST_10.1):")
    print("LCDM chi2 =", float(sumL), "| parts =", partsL)
    print("ECTI chi2 =", float(sumE), "| parts =", partsE)
    print("Δχ² (ECTI−LCDM) =", float(sumE - sumL))
    print("====================================================")


# 14) Reproducibility, data availability, and audit checklist

This notebook is designed to be **fully auditable** and reproducible under a referee-style standard.

---

## A) Model definitions (what is and is not modified)

### ΛCDM baseline
Spatially-flat ΛCDM background with standard radiation content used where required for early-time quantities.

### ECTI-P (late-time extension)
ECTI-P modifies the background expansion **only** at low redshift via \(H(z)\) (see Section 4).
- No recombination changes
- No Boltzmann solver (CLASS/CAMB)
- No TT/TE/EE likelihood, no CLik
- No new growth physics parameters

Growth for RSD is computed using the **standard GR linear growth equation**, driven by the model background \(H(z)\), with no extra degrees of freedom.

---

## B) Parameterization and fairness (same-k)

The sampled parameter vector is identical for both models:
\[
\theta_5 = (H_0,\ \Omega_{m0},\ \sigma_8,\ M,\ \omega_b).
\]

ECTI-P internal parameters \((\beta, z_t)\) are **fixed constants** (not sampled).  
Therefore the comparison is strictly **FAIR**: \(k=5\) vs \(k=5\), implying:
\[
\Delta \mathrm{AIC} = \Delta \mathrm{BIC} = \Delta \chi^2.
\]

---

## C) Data inputs (exact sources)

### Supernovae (Pantheon+SH0ES)
- Files: `Pantheon+SH0ES.dat`, `Pantheon+SH0ES_STAT+SYS.cov`
- The notebook includes an **auto-download** step from the official Pantheon+SH0ES DataRelease if files are missing.

### BAO (DESI 2024)
- Loaded from the public likelihood data repository `CobayaSampler/bao_data`
- Files used:
  - `desi_2024_gaussian_bao_ALL_GCcomb_mean.txt`
  - `desi_2024_gaussian_bao_ALL_GCcomb_cov.txt`
- BAO predictions are evaluated as \(D_V/r_s\) using a **computed** \(r_s(z_\*)\) that depends on \(\omega_b\) (no fixed fiducial \(r_s\)).

### RSD (\(f\sigma_8\))
- Input file: `rsd.csv`
- \(f\sigma_8(z)\) is computed from the standard GR linear growth equation using the model background \(H(z)\) and the sampled \(\sigma_8\).

### KiDS (compressed constraint)
- Implemented as a **compressed Gaussian constraint** on an \(S_8\)-like combination.
- This is **not** the full cosmic shear 2pt likelihood; it is used as an effective summary constraint.

### CMB compressed prior (Planck “lite” 3D)
- Uses the distance-prior vector \((R,\ \ell_A,\ \omega_b)\) with a **3×3 covariance** (as defined explicitly in Section 5.5).
- The mapping uses coherent early-time quantities \(D_M(z_\*)\) and \(r_s(z_\*)\) including standard radiation content and \(\omega_b\).

### Planck lensing derived 1D constraint (optional)
- Implemented as an independent 1D Gaussian constraint:
  \[
  \sigma_8 \Omega_m^{0.25} = 0.589 \pm 0.020,
  \]
  from Planck 2018 lensing-only analysis (see Section 5.6).
- Toggle: `USE_PLANCK_LENSING` (counts as **one** datum when enabled).

---

## D) MCMC implementation

- Sampler: `emcee` v3
- Backend: HDF5 (`emcee.backends.HDFBackend`) for resume-safe runs
- Paired execution policy: LCDM_run_i → ECTI_run_i → report \(\Delta \chi^2\)
- MAP extraction is performed post-burn-in.
- Diagnostics include:
  - total \(\chi^2\)
  - \(\chi^2\) per probe
  - Gelman–Rubin \(\hat{R}\), integrated autocorrelation time \(\tau_{\mathrm{int}}\), and ESS

---

## E) Run-All safety and expected outcome

- Execution is controlled by `RUN_MCMC`.
- When `RUN_MCMC=False`, the notebook will skip post-MCMC blocks safely.
- When `RUN_MCMC=True` and chains exist in `CAMPAIGN_DIR`, running top-to-bottom reproduces:
  - the paired ΛCDM and ECTI-P results (same datasets, priors, and toggles),
  - the final \(\chi^2\) breakdowns and \(\Delta\chi^2\),
  - the publication figures and LaTeX tables generated by the post-processing sections.

---

## F) Audit checklist (referee-grade)

- [ ] Section 1 scope statements match the implemented pipeline (RSD growth, KiDS compressed, Planck compressed, lensing prior).
- [ ] Same-k fairness is explicit: \(\theta_5\) sampled for both models; \((\beta,z_t)\) fixed for ECTI-P.
- [ ] When `USE_PLANCK_LENSING=True`, the lensing block is included in:
  - total \(\chi^2\),
  - \(\chi^2\) breakdown,
  - \(N_{\mathrm{data}}\) used for BIC.
- [ ] All post-MCMC cells are Run-All safe after kernel restart (no implicit state).
- [ ] Repository version has either cleared outputs or avoids leaking machine-local paths.

#End of the notebook